# Project 3 - Search-Based Web Fuzzer 

In [ ]:
import fuzzingbook_utils

## Introduction

Fuzzing web applications can be challenging. A simple web form requires a specific input format for each field, this format needs to be fulfilled for a valid web interaction (e.g. an email field is expected to fulfill the following regex pattern `r'^([a-z0-9_\.-]+)@([\da-z\.-]+)\.([a-z\.]{2,6})$')`.

In this project, we demonstrate how to apply a search-based algorithm to generate specific test inputs for web interfaces. The generated inputs should exercise the web application by fulfilling the input validation requirements of all fields required to reach specific web pages (both normal and error pages). 

The task is to employ a genetic algorithm (GA) for fuzzing, your implementation is expected to generate inputs that match the expected pattern or constraints of the input field (e.g. email fuzzingbook@gmail.com fulfils the expected email regex pattern `r'^([a-z0-9_\.-]+)@([\da-z\.-]+)\.([a-z\.]{2,6})$')`. This is to be achieved by searching the input space starting from an initially (random) input. 

In [ ]:
from WebFuzzer import init_db, SimpleHTTPRequestHandler
from http.server import HTTPServer, HTTPStatus 

### Objective

The goal of this project is to _implement a search-based fuzzing algorithm that generates specifically formatted inputs which fulfil the input validation requirements of a web form_. 

You will apply techniques learned in the lecture ([SBST](SBST.ipynb) and [WebFuzzer](WebFuzzer.ipynb)) to automatically fill web forms by producing inputs that improve the reachability of web pages. 

The goal of your Fuzzer is to cleverly search the input space until the input validation format of the input field is achieved. Consequently, fulfilling (or not fulfilling) such input validation schemes achieves the reachability of specific web pages and improves the coverage of the site map. 

# Web Application

We create a (HTML only) web application for placing an order of items, similar to the order form in the [WebFuzzer](WebFuzzer.ipynb) lecture.


## Input Validation

We create a `ProjectHTTPRequestHandler` class that provides a web order form with input validation schemes for all input formats. 

First, we create a sample set of regex for input validation of each input field in our order form. 

__Note that your implementation shall be evaluated with a _secret set of regex_ other than the ones provided below.__

For some input fields, we have provided other sample regexes you may use to test your implementation, other regexes are provided to inform you of the nature of the regexes which may be used to evaluate your final solution.  


In [ ]:
itemList = [r'^(tshirt|drill|lockset)\Z', r'^(lockset)\Z' ]

name_regex = [r'^[A-Za-z.\'\-\p{L}\p{Zs}\p{Lu}\p{Ll}\']+$', r'^([A-Z]{1}+[a-z\-\.\']*+[\s]?)*$']

email_regex = [r'^([a-z0-9_\.-]+)@(yahoo\.com|gmail\.com|cispa\.saarland)$',\
               r'[a-zA-Z0-9_\.\+-]+\[at\]\[](25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\[dot\](25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\]$']

zip_regex = [r'^[ABCEGHJKLMNPRSTVXY]\d[ABCEGHJ-NPRSTV-Z][ ]?\d[ABCEGHJ-NPRSTV-Z]\d$', \
            r'^GIR[ ]?0AA|((AB|AL|B|BA|BB|BD|BH|BL|BN|BR|BS|BT|CA|CB|CF|CH|CM|CO|CR|CT|CV|CW|DA|DD|DE|DG|DH|DL|DN|DT|DY|E|EC|EH|EN|EX|FK|FY|G|GL|GY|GU|HA|HD|HG|HP|HR|HS|HU|HX|IG|IM|IP|IV|JE|KA|KT|KW|KY|L|LA|LD|LE|LL|LN|LS|LU|M|ME|MK|ML|N|NE|NG|NN|NP|NR|NW|OL|OX|PA|PE|PH|PL|PO|PR|RG|RH|RM|S|SA|SE|SG|SK|SL|SM|SN|SO|SP|SR|SS|ST|SW|SY|TA|TD|TF|TN|TQ|TR|TS|TW|UB|W|WA|WC|WD|WF|WN|WR|WS|WV|YO|ZE)(\d[\dA-Z]?[ ]?\d[ABD-HJLN-UW-Z]{2}))|BFPO[ ]?\d{1,4}$'] 

city_regex = [r'^[a-zA-Z]+(?:[\s-][a-zA-Z]+)*$', r'^\b(\w+)\b(?=.*\1)$']

terms_regex = r'^on\Z'


list_regex = [itemList[0], name_regex[0], email_regex[0], city_regex[0], zip_regex[0], terms_regex]

In [ ]:
print("len name_regex: ", len(name_regex))
print("len email_regex: ", len(email_regex))
print("len zip_regex: ", len(zip_regex))
print("len city_regex: ", len(city_regex))

We define diagnostic error messages for each error page.

In [ ]:
error_msg_list = ["INCOMPLETE INPUT: \n  All fields have to be filled ",\
                   "INVALID INPUT: \n  Item is not in item list",\
                   "INVALID INPUT: \n  Name does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  Email does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  City does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  Zip does not match expected pattern :\n ",\
                  "INVALID INPUT: \n  Terms and Conditions have to be checked "
                  ]

We override the `handle_order()` method, to validate the values of each input field using the pre-defined regex patterns above. 

We use the [regex parser](https://pypi.org/project/regex/), specifically version 2.5.23, instead of Python's native `re`, because `re` does not support the `\p{}` syntax. Confirm regex version using: 

In [ ]:
!pip install regex 
import regex
print(regex.__version__)

In [ ]:
class ProjectHTTPRequestHandler(SimpleHTTPRequestHandler):
    def handle_order(self): 
        values = self.get_field_values()

        if len(values) < 6:
            error_type = error_msg_list[0]
            self.internal_server_error(error_type)
            
        elif regex.match(list_regex[0], values["item"]) and regex.match(list_regex[1], values["name"]) and \
        regex.match(list_regex[2], values["email"]) and regex.match(list_regex[4], values["zip"]) and \
        regex.match(list_regex[3], values["city"]) and regex.match(list_regex[5], values["terms"]):
            self.store_order(values)
            self.send_order_received(values)

        elif not regex.match(list_regex[0], values["item"]):
            error_type = error_msg_list[1]
            self.internal_server_error(error_type) 
        
        elif not regex.match(list_regex[1], values["name"]):
            error_type = error_msg_list[2] + list_regex[1] 
            self.internal_server_error(error_type)
        
        elif not regex.match(list_regex[2], values["email"]):
            error_type = error_msg_list[3] + list_regex[2]
            self.internal_server_error(error_type)
            
        elif not regex.match(list_regex[3], values["city"]):
            error_type = error_msg_list[4] + list_regex[3]
            self.internal_server_error(error_type)
        
        elif not regex.match(list_regex[4], values["zip"]):
            error_type = error_msg_list[5] + list_regex[4]
            self.internal_server_error(error_type)
    
        elif not regex.match(list_regex[5], values["terms"]):
            error_type = error_msg_list[6]
            self.internal_server_error(error_type)
        

### Internal Errors
For diagnostic (and identification) purposes, the internal server error pages include the error message indicating the input validation scheme that failed.

In [ ]:
HTML_INTERNAL_SERVER_ERROR = """
<html><body>
<div style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">
  <strong id="title" style="font-size: x-large">Internal Server Error</strong>
  <p>
  The server has encountered an internal error.  Go to our <a href="/">order form</a>.
  <pre>{error_message}</pre>
  </p>
</div>
</body></html>
  """

In [ ]:
import sys
import traceback

class ProjectHTTPRequestHandler(ProjectHTTPRequestHandler):
    def internal_server_error(self, error_type=None):
        self.send_response(HTTPStatus.OK, "Internal Error")

        self.send_header("Content-type", "text/html")
        self.end_headers()

        exc = traceback.format_exc()
        self.log_message("%s", exc.strip())

        message = HTML_INTERNAL_SERVER_ERROR.format(error_message=error_type)
        self.wfile.write(message.encode("utf8"))

In [ ]:
from fuzzingbook_utils import HTML 
HTML(HTML_INTERNAL_SERVER_ERROR)

### Storing Orders

Similar to the [WebFuzzer](WebFuzzer.ipynb) lecture, we store orders using a *database* stored in the file `orders.db`.

In [ ]:
FUZZINGBOOK_SWAG = {
    "tshirt": "One FuzzingBook T-Shirt",
    "drill": "One FuzzingBook Rotary Hammer",
    "lockset": "One FuzzingBook Lock Set"
}

In [ ]:
import sqlite3, os

ORDERS_DB = "orders.db"

class ProjectHTTPRequestHandler(ProjectHTTPRequestHandler):
    def store_order(self, values):
        db = sqlite3.connect(ORDERS_DB)
        db.execute("INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?)",
                (values['item'], values['name'], values['email'], values['city'], values['zip'], values['terms']))
        db.commit()

    def send_order_received(self, values):
        values["item_name"] = FUZZINGBOOK_SWAG[values["item"]]
        confirmation = HTML_ORDER_RECEIVED.format(**values).encode("utf8")

        self.send_response(HTTPStatus.OK, "Order received")
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(confirmation)
    

In [ ]:
def init_db():
    if os.path.exists(ORDERS_DB):
        os.remove(ORDERS_DB)

    db_connection = sqlite3.connect(ORDERS_DB)
    db_connection.execute("DROP TABLE IF EXISTS orders")
    db_connection.execute("CREATE TABLE orders (item text, name text, email text, city text, zip text, terms text)")
    db_connection.commit()

    return db_connection

db = init_db()

In [ ]:
print(db.execute("SELECT * FROM orders").fetchall())

db.execute("INSERT INTO orders " +
           "VALUES ('lockset', 'Jane Lee Doe', 'jane@doe.com', 'Saarbrucken', '66125', 'on')")
db.commit()

In [ ]:
print(db.execute("SELECT * FROM orders").fetchall())


db.execute("DELETE FROM orders WHERE name = 'Walter White'")
db.commit()

In [ ]:
print(db.execute("SELECT * FROM orders").fetchall())

### Order Confirmation

Once we have gotten an order, we show a confirmation page, which is instantiated with the customer information submitted before.

In [ ]:
HTML_ORDER_RECEIVED = """
<html><body>
<div style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">
  <strong id="title" style="font-size: x-large">Thank you for your Fuzzingbook Order!</strong>
  <p id="confirmation">
  We will send <strong>{item_name}</strong> to {name} in {city}, {zip}<br>
  A confirmation mail will be sent to {email}.
  </p>
  <p>
  Want more swag?  Use our <a href="/">order form</a>!
  </p>
</div>
</body></html>
"""

### Logging 

Similar to the [WebFuzzer](WebFuzzer.ipynb) lecture, we provide a logging infrastructure for debugging purposes. We implement the following:
- `print_httpd_messages()` prints all messages accumulated in the queue
- `clear_httpd_messages()` discards all pending messages
- `log_message()` store messages in the queue 

In [ ]:
from multiprocessing import Queue, Process
from fuzzingbook_utils import rich_output, terminal_escape, unicode_escape, HTML

HTTPD_MESSAGE_QUEUE = Queue()

def display_httpd_message(message):
    if rich_output():
        display(
            HTML(
                '<pre style="background: NavajoWhite;">' +
                message +
                "</pre>"))
    else:
        print(terminal_escape(message))

def print_httpd_messages():
    while not HTTPD_MESSAGE_QUEUE.empty():
        message = HTTPD_MESSAGE_QUEUE.get()
        display_httpd_message(message)

def clear_httpd_messages():
    while not HTTPD_MESSAGE_QUEUE.empty():
        HTTPD_MESSAGE_QUEUE.get()

class ProjectHTTPRequestHandler(ProjectHTTPRequestHandler):
    def log_message(self, format, *args):
        message = ("%s - - [%s] %s\n" %
                   (self.address_string(),
                    self.log_date_time_string(),
                    format % args))
        HTTPD_MESSAGE_QUEUE.put(message)

Extend `webbrowser()` method to prints log messages produced by the server:

In [ ]:
from Carver import webbrowser as simple_webbrowser

def webbrowser(url, mute=False):
    try:
        contents = simple_webbrowser(url)
    finally:
        if not mute:
            print_httpd_messages()
        else:
            clear_httpd_messages()

    return contents

## Running the Web Application
Similar to the [WebFuzzer](WebFuzzer.ipynb) lecture, to run the web application we implement the following:

* `run_httpd_forever()`
* `start_httpd()`
* `print_url()`


In [ ]:
def run_httpd_forever(handler_class):
    host = "127.0.0.1"  # localhost IP
    for port in range(8800, 9000):
        httpd_address = (host, port)

        try:
            httpd = HTTPServer(httpd_address, handler_class)
            break
        except OSError:
            continue

    httpd_url = "http://" + host + ":" + repr(port)
    HTTPD_MESSAGE_QUEUE.put(httpd_url)
    httpd.serve_forever()

def start_httpd(handler_class=ProjectHTTPRequestHandler):
    clear_httpd_messages()

    httpd_process = Process(target=run_httpd_forever, args=(handler_class,))
    httpd_process.start()

    httpd_url = HTTPD_MESSAGE_QUEUE.get()
    return httpd_process, httpd_url

httpd_process, httpd_url = start_httpd(ProjectHTTPRequestHandler)

def print_url(url):
    if rich_output():
        display(HTML('<pre><a href="%s">%s</a></pre>' % (url, url)))
    else:
        print(terminal_escape(url))

print_url(httpd_url)

**Note**: The above URL only works if you are running the Jupyter notebook server on the local host.

## Testing the Web Application
### Input Format

When the user clicks `Submit` on the order form, the Web browser creates and retrieves a URL of the form:

```
<hostname>/order?item=value_1&name=value_2&email=value_3&city=value_4&zip=value_5&terms=value_6
```

We test standard behavior:

In [ ]:
standard_order = "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40cispa.saarland&city=Seattle&zip=A1A1A1&terms=on"
contents = webbrowser(httpd_url + standard_order)
HTML(contents)

In [ ]:
assert contents.find("Thank you") and contents.find("We will send") and contents.find("A confirmation mail will be sent to") > 0

We test erroneous behavior - terms and conditions not checked:

In [ ]:
error_order = "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40cispa.saarland&city=Seattle&zip=A1A1A1&terms=off"
contents = webbrowser(httpd_url + error_order)
HTML(contents)

In [ ]:
assert contents.find("Internal Server Error") and contents.find(error_msg_list[6]) > 0

We test incomplete order behavior:

In [ ]:
incomplete_order = "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104"
contents = webbrowser(httpd_url + incomplete_order)
HTML(contents)

In [ ]:
assert contents.find("Internal Server Error") and contents.find(error_msg_list[0]) > 0

To test for another input validation regex (e.g. the second regex pattern for each field - index 1), use:

### Obtaining Coverage

We define the `target_list` for each web page accordingly:

In [ ]:
"""
target_list:
    ISE_Name = Internal Server Error Web Page for "Name" field error, analogously for ISE_Item, ISE_City, etc..
    ISE_INC = Internal Server Error Web Page for "Incomplete Input" 
    PNF =  Page Not Found Web Page
    CONF = CONFIRMATION_PAGE
"""
target_list = ["ISE_INC", "ISE_Item", "ISE_Name", "ISE_Email", "ISE_City","ISE_Zip", "ISE_Terms", "CONF", "PNF"]

To obtain the overall coverage achieved by the GA's resulting population, we implement the following web reachability function. 

For each target, we compute:
* _validity_ score: the number of valid inputs in the generated population that reach the target, and
* _reachability_ score: we score the number of inputs that reach the each page. 

The reachability scores for the number of inputs reaching the target is based on the perceived difficulty to reach the page:
* 2 points for each input reaching an error page
* 5 points for each input reaching the confirmation page
* 1 point for each input reaching the page not found page

**Note**: we expect each run of your genetic algorithm to produce 10 inputs maximum.

In [ ]:
def get_web_reachability(orders, target):
    global httpd_process, httpd_url
    num_reached_pages = dict() 
    reachability, validity = 0, 0
    conf_msg = ["Thank you", "We will send", "A confirmation mail will be sent to"]
    page_not_found = "This page does not exist"
    
    for order in orders:
        try:
            clear_httpd_messages()
            contents = webbrowser(httpd_url + order, mute=True)
            HTML(contents)
            
            # obtain 2 reachability points for each input reaching an error page
            if contents.find("Internal Server Error") > 0:
                if target_list.index(target) <= 6:
                    msg = error_msg_list[target_list.index(target)]
                    if contents.find(msg) > 0: 
                        if not msg in num_reached_pages:
                            num_reached_pages[msg] = 2
                        else:
                            num_reached_pages[msg] += 2
                        validity += 1

            # obtain 5 reachability  points for each input reaching confirmation page
            if target ==  target_list[7] and contents.find(conf_msg[0]) > 0 and contents.find(conf_msg[1]) > 0 \
            and contents.find(conf_msg[2]) > 0:
                if not conf_msg[0] in num_reached_pages:
                    num_reached_pages[conf_msg[0]] = 5
                else:
                    num_reached_pages[conf_msg[0]] += 5
                validity += 1

            # obtain 1 reachability  point for each input reaching a non-existent page
            if target == target_list[8] and contents.find(page_not_found) > 0:
                if not page_not_found in num_reached_pages:
                    num_reached_pages[page_not_found] = 1
                else:
                    num_reached_pages[page_not_found] += 1
                validity += 1
            
        except Exception as e:
            e.__traceback__ = None
            traceback.clear_frames(e.__traceback__)
            pass
            
        
    if len(num_reached_pages) > 0:
        reachability = sum(num_reached_pages.values())
    return reachability, validity

We test the `get_web_reachability` function for the `confirmation` page target with a sample list of test order URLs. 

In [ ]:
test_orders = ["/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=off",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order", "/none",\
                  "/order?item=lockset&name=Jane+lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40cispa.saarland&city=Seattle&zip=98104&terms=on",\
                  "/order?item=book&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=98104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Seattle&zip=498104&terms=on",\
                  "/order?item=tshirt&name=Jane+Lee+Doe&email=doe%40example.com&city=Neunkirchen&zip=98104&terms=on",\
                 ]

Test for the reachability of confirmation page ("CONF") using the above order inputs:

In [ ]:
get_web_reachability(test_orders, "CONF")

## Search Based Fuzzing

We demonstrate how a specific input format can be generated with a simple search-based algorithm and a naive fitness function. 

We assume an input url with an invalid name format error:

In [ ]:
import random, string
invalid_name_order = "/order?item=tshirt&name=Jane+Lee+Doe1&email=doe%40example.com&city=Seattle&zip=98104&terms=on"
contents = webbrowser(httpd_url + invalid_name_order)
HTML(contents)
assert contents.find("Internal Server Error") and contents.find(error_msg_list[2]) > 0

In [ ]:
import urllib.parse
test_name = urllib.parse.parse_qs(invalid_name_order, keep_blank_values=True)["name"][0] 

We implement a naive distance function for "name" input field:

In [ ]:
def naive_calculate_name_distance(name):
    name_regex = r'[A-Z][a-z]+ [A-Z][a-z]+ [A-Z][a-z]+'
    res = 1
    if len(name.split()) == 3 and check_ascii_letters(name) and \
        check_word_size(name) and name.istitle():
        res = 0
    else:
        if len(name.split()) == 3:
            res -= 0.4
        if check_ascii_letters(name):
            res -= 0.35
        if check_word_size(name):
            res -= 0.15
        if name.istitle():
            res -= 0.10
    return res 

def check_ascii_letters(name):
    res =  True
    for char in name.replace(" ", ""):
        if not (char in string.ascii_letters):
            res = False
            break
    return res

def check_word_size(name):
    res = True
    for word in name.split():
        if len(word) < 2:
            res = False
            break
    return res

We implement a naive fitness function for the name input. 


In [ ]:
def get_naive_fitness(name):
    global distance
    distance = naive_calculate_name_distance(name)
    fitness = distance
    return fitness

We implement a simple mutation function `flip_random_character` for the name input. 

In [ ]:
def flip_random_character(s):
    pos = random.randint(0, len(s) - 1)
    new_c = chr(random.randrange(65, 130))
    return s[:pos] + new_c + s[pos + 1:]

We implement a simple search-based algorithm -`randomized_hillclimber()`, to run for a maximum of 5000 iterations :

In [ ]:
def randomized_hillclimber(name):
    fitness = get_naive_fitness(name)
    print("Initial value: %s at fitness %.4f" % (name, fitness))
    iterations = 0

    while fitness > 0 and iterations < 5000:
        iterations += 1
        
        mutated = flip_random_character(name)
        new_fitness = get_naive_fitness(mutated)
        if new_fitness <=  fitness:
            name = mutated
            fitness = new_fitness

    if iterations < 5000:
        print("Optimum at %s after %d iterations" % (name, iterations))
    else:
        print("# iterations  is: %d, stop!!!" % (iterations))
    return name, iterations

In [ ]:
for i in range(0, 5):
    new_name, no_iterations = randomized_hillclimber(test_name)   
    if no_iterations < 5000:
        new_name = new_name.replace(" ", "+")
        new_order = "/order?item=tshirt&name=" + new_name + "&email=doe%40cispa.saarland&city=Seattle&zip=A1A1A1&terms=on"
        new_order
        break
    new_name = new_name.replace(" ", "+")
    new_order = "/order?item=tshirt&name=" + new_name + "&email=doe%40cispa.saarland&city=Seattle&zip=A1A1A1&terms=on"
    new_order

In [ ]:
new_order

In [ ]:
clear_httpd_messages()
contents = webbrowser(httpd_url + new_order)
HTML(contents)

In [ ]:
assert contents.find("Thank you") and contents.find("We will send") and contents.find("A confirmation mail will be sent to") > 0

# Genetic Algorithm

## Genetic Algorithm template

The basic template of the genetic algorithm is based on the [SBST](SBST.ipynb) from the lecture.

This template uses your implemented fitness, selection, mutation and crossover functions for the population of input urls. 

In [ ]:
from multiprocessing import Process
import traceback

def run_with_limited_time(func, args, kwargs, time):
    p = Process(target=func, args=args, kwargs=kwargs)
    p.start()
    p.join(time)
    if p.is_alive():
        p.terminate()
        return False

    return True


In [ ]:
def evaluate_population(population, target, list_regex):
    fitness = [get_fitness(x, target, list_regex) for x in population]
    return list(zip(population, fitness))

In [ ]:
def terminal_repr(s):
    return terminal_escape(repr(s))

In [ ]:
import time

def genetic_algorithm(population, target, list_regex, return_dict):
    # Generate and evaluate initial population
    generation = 0
    fitness = evaluate_population(population, target, list_regex)
    best = min(fitness, key=lambda item: item[1])
    best_individual = best[0]
    best_fitness = best[1]

    start_time = time.time()
    time_spent = 0
        
    # Stop when optimum found, or we run out of patience, i.e. 1000 iterations or 1 hour max
    while best_fitness > 0 and generation < 1000 and (time_spent <= 3600):

        # The next generation will have the same size as the current one
        new_population = []
        while len(new_population) < len(population):
            
            # Selection
            try:
                if run_with_limited_time(selection, (fitness, 10,), {}, 2):
                    offspring1 = selection(fitness, 10)
            except Exception as e:
                e.__traceback__ = None
                traceback.clear_frames(e.__traceback__)
                pass
            
            try:
                if run_with_limited_time(selection, (fitness, 10,), {}, 2):
                    offspring2 = selection(fitness, 10)
            except Exception as e:
                e.__traceback__ = None
                traceback.clear_frames(e.__traceback__)
                pass

            # Crossover
            if random.random() < 0.7:
                try:
                    if run_with_limited_time(crossover, (offspring1, offspring2,), {}, 2):
                        (offspring1, offspring2) = crossover(offspring1, offspring2)
                except Exception as e:
                    e.__traceback__ = None
                    traceback.clear_frames(e.__traceback__)
                    pass

            # Mutation
            try:
                if run_with_limited_time(mutate, (offspring1,), {}, 2):
                    offspring1 = mutate(offspring1)
            except Exception as e:
                e.__traceback__ = None
                traceback.clear_frames(e.__traceback__)
                pass
            
            try:
                if run_with_limited_time(mutate, (offspring2,), {}, 2):
                    offspring2 = mutate(offspring2)
            except Exception as e:
                e.__traceback__ = None
                traceback.clear_frames(e.__traceback__)
                pass
            

            new_population.append(offspring1)
            new_population.append(offspring2)

        # Once full, the new population replaces the old one
        generation += 1
        population = new_population
        
        try:
            if run_with_limited_time(evaluate_population, (population, target, list_regex,), {}, 10):
                fitness = evaluate_population(population, target, list_regex)
        except Exception as e:
            e.__traceback__ = None
            traceback.clear_frames(e.__traceback__)
            pass

        best = min(fitness, key=lambda item: item[1])
        best_individual = best[0]
        best_fitness = best[1]
        
        
        end_time = time.time()
        time_spent = end_time - start_time
        
    return_dict[target] = population, generation

## Your Tasks
For each input, you are epxected to produce a set of urls to reach a targeted web page, i.e. generate inputs that will fulfill the input validation requirements necessary to reach a specific web page. Overall, all web pages will be targets, i.e. both error and normal pages. 

Your task is to implement your own custom selection, fitness and mutation functions for the genetic algorithm, in order to fulfill the input validation and web reachabilty requirements.

## Tips

* Your fitness function should target a specific web page,e.g. a specific error page like an invalid name error page. 
* For input (fragment) distance consider using a (fuzzy/approximate) regex matching distance of the input to the expected regex.
* You are allowed to use the `fuzzingbook` module, any of the [inbuilt python modules](https://docs.python.org/3/py-modindex.html) and the [regex module](https://pypi.org/project/regex/). 
* __Do not import external modules other than the afore-mentioned modules, and do not use (parts of) open source code__.
* You can develop any type of mutation, selection and crossover functions.
* Ensure that your implementation accounts for any arbitrary regex and any random initial population of inputs.
* For debugging purposes: unmute the `webbrowser()` to obtain logging information, i.e. set `webbrowser(url, mute=False)` 
* Do not implement in any other section except the section below. 
* Gracefully handle exceptions and errors resulting from your impelementation.
* __Remember the input validation regex and initial input population could be arbitrary, do not hard code for a specific regex or input__.

## Your Code

In the following section implement the following:
* __Fitness Function(s)__:  define a (set of) fitness function for an order URL to validate/invalidate each input field to reach specific pages (target). 

* __Selection Function__: define a selection function for the given generated population of URLs.

* __Crossover Function__: define a crossover function for two parent input URLs.

* __Mutate Function__: define a mutate function for our input URL.

Implement your own fitness, selection and mutation functions for the GA algorithm below:

In [ ]:
"""
INSERT YOUR CODE BELOW
"""

# Evaluation code

The code in the following section will be used to evaluate your impelementation.

## Random Population of Inputs
We randomly generated an `initial_population` of 20 set of inputs for our order web form, with (4) varying levels of difficulty (or garbage). Each populations of input URL contains 10 input URLs for orders.

__Note that the url path population which will be used for evaluation may be different from those provided as example.__

In [ ]:
"""
Final Evaluation Population 
"""

easy_population_1 = ['/order?item=drills&name=[Jc{_kWmegnEkeAGtSLBELU]uGA^rBiBqG{r]N`IoetX+dLC]d{edp]GvxcxMBvb[jV{WW`Ihy}mdoWAKj_XnFRwMF`PJ+`k{mtsa]p]zxqbI^tJE`J~_k}[C&email=rmbhbgudcludejtmmpirjn%40mdx&city=V{Tl~&zip=77011&terms=checked',
 '/order?item=drills&name=zxBjCtPVYywPxTdHff\\`HAWJYj[]+f{[NUQ{GusHnC{B_z`JfbQ|~opk~uZOW]ml}iPqzf`JWObf+vsb]BUKpv}OyOCCTsLw&email=wrwgyudtzzugpiqbxkvpsdnsn%40ffhzk&city=qBHizzZUc&zip=0051&terms=undefined',
 '/order?item=lockset&name=CJvhKs+dlTOcfhCxXqN{+Rf_getneyFKVbyDpmcRBFqK~Von^{|EhlFZjkixVzAQkvJ&email=vnadrrlxdxbumxh%40pvdr&city=ryVOms&zip=04381&terms=on',
 '/order?item=Drill&name=DDLIoB|SlUNLjhd\\maS[gfYyCtL{s+VPJlNEpJ`CKzEjVd^tT~`JQlO_UEzXKOWsoLh_phRl{TsyOi+smi_Pcmaai\\r]giveM_heEs}A~fCRjd_EAue\\WviMHi&email=zeeznfn%40jrth&city=VI&zip=57548&terms=checked',
 '/order?item=tshirts&name=N{CiIzVPusVZbhBMBgmJJuGz|a{LASwJoio]Cd{F}x^+}iVDDK{`oruK\\[jfSiKaE~+k`kZTjlFxUioOGsyKGFS][JpR}d|HszWF]MVTEblLBiLZE&email=kesqhzlvhlximqjbccgiyzncpxj%40nbw&city=UDTXRB&zip=33811&terms=off',
 '/order?item=tshirt&name=pgn{LAtPbybM^iKCsaqoyJTTlqgj|Nlj}JqBke+gH\\_wiCw`hbifqv^kZIQQNnmB{}+Oy~mASjADyscGkHRIjeHCyOqJWzduuUY^cvjxloyEzLAjjB|^&email=mgbzugwtrrfhxxzmrho%40mgrw&city=n\\fTDv&zip=77430&terms=checked',
 '/order?item=lockseR&name=CtZ\\nmdILfq+Mr\\MT|TWgU]nlm`CsH|plOiQlqep+Nx|X`ATl&email=ynddkdata%40iofw&city=gRSFaG&zip=20425&terms=checked',
 '/order?item=drills&name=WCeAK[dalkWlb|o+~^gyG|bPob^^CyD{uhfGXhKGuBDFFNO+wiTjUweb}SpWlqPCcqFZcw`WapU&email=xpxc%40xgr&city=|kX}YdbS&zip=2115&terms=on',
 '/order?item=Tshirt&name=pH^O_JkIXXiGfM{f}w+`E`GRZkSj`pLs}CJ+wc}SxgcjVULzn{\\KoYsa}PVYWosNtl[_bDjUBIM&email=gvqsbvybsqvyyqeymaayhygeey%40mm&city=wvX`ejL&zip=1737&terms=on',
 '/order?item=drill&name=t{FYfvEISENrA{oX`SuL}zRuk+VVPAp\\kI]~VV`|FGv+XxwrB^CPrmA&email=qqabjqecyyrebipafywb%40sl&city=AM^[&zip=3572&terms=checked']

easy_population_2 = ['/order?item=tshirts&name=jyqSzuLrz]iNPnPnabk{tDl|UHrYyyYsW~O[}npUbR+AP|yOVW|a}wWVlWBUVB{jgZZ[z`Q+agpxKdcHUhhR|z}mhSLubCP&email=mgfzfcbswarlq%40pnnb&city=kJ&zip=81082&terms=checked',
 '/order?item=tshirts&name=upcsILr~Q~OswbduJDNHVqEj\\[pFad|xbgrh{yiZ[O~K^ZB+jk~eiQ~lvnr_lOPn+GqVJmLEQtavXM`rmXhbEsOvC[b|[LXRDr`ZvqVI&email=jszvmovdeszjixxrqkgvl%40yz&city=lXqDREY&zip=38824&terms=undefined',
 '/order?item=tshirts&name=iV_[ub|^gT^Eb+x]dQ~zgyHo|[|RHtO[d+RngZ}CY{QzBghqMtIq\\e\\kyCqPMqfS_Uhs[t&email=ckgowplsmqxizngylvlozlpwajbb%40zievm&city=rSZkK&zip=7841&terms=off',
 '/order?item=Drill&name=^yPjoCqJaotBNh_zdKeXbry`Tj{_uV[sTa}a+`oornirHtmvZR[rQ{v[kWzR_yWLXmnFMuamkWSKljuZZr+|DK]C|YTXiSsLyigbNnPkWC}pJd}~TBh}erh&email=ogbnfdbdziafbaszhktnam%40mozw&city=KGklxeWR&zip=1860&terms=on',
 '/order?item=lockseR&name=}yrZkmLMeNLq`tv\\NAE|+\\EvE`LvTIwljOaw^FE{LTH]HtSP]aswb[Vtf\\yUGErO|WrE+sxuTSqt_Dqk`h}dptYBWQNMC~gpS^JH~rXTtMbUqmlkfwQg&email=bfmiirhnfexlo%40da&city=hWymOn_uV&zip=70770&terms=undefined',
 '/order?item=lockSet&name=QuhI~Q\\QKx_rrPuihA|BGyFaTr+XfGsd]K`I{ChyD~he+B~kW~psGF|uBfj|_MQGBxp_QOA|c&email=jskkldilypp%40ksgkt&city=IsxJ&zip=4120&terms=undefined',
 '/order?item=Tshirt&name=aoyTkw|Djhj{wC+uBxmQEP`C~\\n`cxgTj~ZNURdQc\\EbHlhgoxAswBifEJ+DYsuhk]SFz|ZcarS[IwuJKkgu\\fxU~_ZcSjpDSW[&email=aaziuncgowqzms%40my&city=UjtbtZJno&zip=12448&terms=off',
 '/order?item=lockset&name=GRceQw+u^jsV]vezGy{hSk`gZnMPKlzOQHG{SGOaJ+`~ddDKxYIeUEyVWURdYVtonkQSrbLZi&email=myryaifhoguezpaaeifjqbqxadso%40jinx&city=vGK&zip=4143&terms=off',
 '/order?item=drill&name=bytJ`NvVPjxXMB_bxF`CmvzxotHdIFWJr+}vGZU^lkmQxfyYZz+U`Rdvgxe\\IPyEkgFYXnvmnoYC|_Bc}xkhMFxn}I}~dJhte&email=icwld%40nzjr&city=P\\FpLwjU&zip=7036&terms=undefined',
 '/order?item=tshirts&name=JsIvB{pT\\LhHyI`F_kpfFoeTwGjW^Ah|cHOgnENQKy{{tr+ZXezy^bL}zAlTb^VP}ppCGfAu_hONu^ekOwJDg\\SW`Br+{b[{mCCsNmnRlP~k[B[sMX^DLbE^[aikyRi[kBZFSrorPcR]&email=hdnsshlkjpxkptyurtd%40lncyj&city=u^P&zip=1462&terms=off']

easy_population_3 = ['/order?item=lockset&name=jeBAuzgCQBCPOKkBNKO[pSGBSdlxp\\tIW]+{nL]aX\\x`zJiVdsTAkcEeCO]EmTonmnmJL{+VsSDk^nW]Oj~ct|[JfmEvzoI^S&email=gzlijnpirchhavgufpkkb%40olprc&city=w\\y&zip=1532&terms=undefined',
 '/order?item=tshirt&name=[UOdGfc~COPBKNl`grGIPX^fIYJu+Hr\\j]bVFLLj\\|_gPbaqa[Btgx~MR\\~SHMc+ZJl{E`Yzqd{jdHJN|]NNcmryGT{\\PhE[RRjNeXM&email=rifdnmmayojpixjswyzeoncw%40sama&city=QR&zip=3431&terms=on',
 '/order?item=lockset&name=wIu}Dq+Pyhb]SvOohQZvRbtwp~~xtb]QLkbd`TxACuVRt+nRxYDJzVUosE{H{WKzsvJ&email=dbpnnhe%40zj&city=as&zip=3277&terms=on',
 '/order?item=drill&name=LKCODOKtdum+gm~gQKH}m_vay}wt^COcoD~z|ZhPT~Q`+Pdqffn}z|MZujYneIiWoPtNAPNFScrlIJ|emSR}{g&email=dhqouyp%40vmiy&city=EvuI^R&zip=2174&terms=checked',
 '/order?item=tshirts&name=Lg\\zSvsyKs{K\\~THi{Av~wWW{kzkgfZzvCft_dsxJ+D{Xzb}ZZh^wrMNzKTZHY\\uM[gL~ccPnvFL^zL+RrJ`E~Xd_b]M`NFgP&email=xzkmrojdcgvfmqfruzfrhqwqm%40jn&city=H}Rb&zip=6874&terms=on',
 '/order?item=lockset&name=kANS]xfFAIyY+ll|EE]WE+tZxDK~oQ[kWCo~fFQD{&email=wrcsyvqywvfcvzmkfeorxdieknlv%40gwog&city=Q~x&zip=26836&terms=checked',
 '/order?item=lockseR&name=zSC~K+OFQHFOaoE\\}gtHJbYBiLjLxYQoLd_X|BhQ+sI\\\\^\\Wt\\IUnicyJChzVoDF|VfnygrGMGITyvd&email=rxmbjfcaymadv%40wfv&city=ADWvzJ|f&zip=4256&terms=undefined',
 '/order?item=lockset&name=q`Zeffjyo{yWnSUvr|vr~WDktnK~Yj\\teW+slbPLe`rHxbdTMlCeM`^ZSIx}mSdaM]PSYcxeIUwiB+Xk]`dyLXA_zf~}itaBQvPd|PHwGe~&email=bwvguxlgquqkyjsljtbckgczkxvq%40exsum&city=jc`ggDch`&zip=8216&terms=on',
 '/order?item=drill&name=TMHOIiC^SPdB|UWDJN|a[ErV_zKR+}FPq]F{h``ok`w^RkvYyKMdbQnUJ\\QRkSmHd]}VW|+Cef[miTOYE&email=webzcfhbayiqslodbfshnbtcnqkut%40wnkt&city=]~|dauE&zip=6442&terms=undefined',
 '/order?item=drills&name=kMaiRRVXsaXBbKOuDmI+Zy]ue_SxnTNcgievkK_+Oono|uR]Kvax{MLvMbjJyaE`l_XnjFvMIS&email=oijzvennraqwzayfvzkqxbcbz%40wc&city=pFER`e|h&zip=16668&terms=on']

easy_population_4 = ['/order?item=Tshirt&name=vNUwFBAoyAT+oZ_sm~N_E\\V[qeM`TDiGbhS`QgZYhvZVow+zgiR[BJMESzT_Q_W`tOLFoSdP|ggyouSzUGrOnvVaswceS&email=ewgwntvibdrztrdwmuthfsp%40zv&city=nBPxcVf\\&zip=20734&terms=on',
 '/order?item=tshirts&name=MwljgKvHGWLa|jt~K_+kn^fCwDhfoqcSa\\}WsBm|hmkoOwDJF}K\\T\\h^nU+RVcqfEHeQCMEf}ZGn{gcjxNX|vI|}SU&email=migbpfryy%40yiqzk&city=fl&zip=55703&terms=off',
 '/order?item=tshirt&name=M^PPvbjohBmY~lP{aXk]Hi+}efWl_UTFw[rH[Hco_qLW{+K^K}_gf[MEw^iprEyUAj_NnzcE}pr{hgeRzXyJ&email=ayscfyqtmkefnjrxrgr%40qp&city=vvGjZJ&zip=1133&terms=checked',
 '/order?item=drills&name=`qPZcXZcy_bPShz`|qoRGez_wkJmaApX~wwzpi[]]+NQ[ANDZA\\cBvAfVJ}y^UTXYmo^sf|SoKUPPv]`i+jtvTptCaFbSM`S&email=wuvqyclw%40vtflj&city=]Gs{eG&zip=6628&terms=off',
 '/order?item=lockset&name=ZGlUzW}Huwdpsu`JoFhmo~GKic]jpEOG~+BvDVsxZLA^tvJoxphFM+ueToMlNh&email=nfxm%40ot&city=DjCfLp&zip=4132&terms=checked',
 '/order?item=drill&name=uD][V+ADSY_bFc[wG^We+]qc`AakXb{aPdc}YjHuKbJ|^ctcfdsq^eOuz^DyBzjo&email=ppbmnnq%40jhdkw&city=TKXy|vJT_&zip=87376&terms=on',
 '/order?item=lockSet&name=DOMvOOQVKxLSbQ^AxOXJXrpjUgiym+BWVwgYzAOgsDTIOwS|SPbgQrICM~soqJCeOzdDI\\h+SgPZkLp]^q__VNMNI]~uVQqV^z&email=cbvduqcgsyq%40zj&city=Zk}~zA&zip=77175&terms=undefined',
 '/order?item=tshirts&name=vqPA_gOwWU`jAHi{JWwD[gX|m}[H+ogrJAbsTAp~~g_ZcqLfZKhyIqdUFfFmzPebwHfAoJ|iZNNM+IKTVzwW^oNq~VOfgpOULUEzobC|&email=hcvb%40vjc&city=Ru&zip=23406&terms=on',
 '/order?item=tshirts&name=ojbDYmz]eNe\\nAIwllU{F]fZSRM^MExQgZQtTOy_DEJbcSDn+^J]A\\^Ry`S^gnWK`trokGKExBBupZtzGNmBPIn_KwC+zx_bz&email=baaq%40dy&city=uJ&zip=0534&terms=undefined',
 '/order?item=tshirts&name=]VYBcvLR`ts[iaM{y[R]Z[ZtPnaKufwkSeg+heGEcfP+wtZLWLU&email=zsrzop%40ap&city=XsUrWw&zip=3624&terms=checked']

easy_population_5 = ['/order?item=drill&name=FQdnpJbweAtSsGSPYZayD]JNVvKl[vQpfHwTGeOW}~tH+G|hAeTX~lj\\`ttVlt[[qOm]BDRrzBjBkEfmsOSThaAM+FinRR]UaYhY|qzGgqGSIcfzJnEl}[HL{JHbK&email=nyffowiojoc%40rta&city=|VlMDI&zip=87744&terms=on',
 '/order?item=tshirts&name=Vt}lTM~TTWoiDa`XRDqgmOmzWbSYq{t{Sy+V_sswVHhsS+NmiJnKekBdIG[kk}BR_PPgZ&email=pfdi%40euthk&city=bi&zip=7710&terms=on',
 '/order?item=drill&name=nBblOQGqSEN\\VRdF|+w_vAHOrp+_VKMfP&email=jmpbbbejepvzwpsthrgieo%40vambs&city=P|uFjK^&zip=0563&terms=on',
 '/order?item=tshirts&name=hJe]u[NHvyMeH^[+eRvR`b{j_yKmWJun{BFpxy\\EQctgl{maLorCEGoE+GSrzJVvNTxm}[~OnoBez&email=nzmijelcdfxbemdnhqnpcyoqb%40oeql&city=wEuJXO}^E&zip=3610&terms=checked',
 '/order?item=lockSet&name=aJYAEtOfkrI[DKFcY^+BU_bGmOMPZkJf_hHLsjs+lwJ}up\\&email=mbwelwhftruh%40pt&city=f\\RAYzhnT&zip=5046&terms=undefined',
 '/?item=lockset&name=BLr|kgu[euunEoovRm^i_n&email=U^Oe&city=_p_U`Vw`F{k`OozIw[wRe&zip=IPvAQ&terms=Y[S',
 '/?item=drills&name=Yz`bKiAgsUBinT`eE\\~ug\\~thWB{ECEZfy{znh`{heQ&email=YkTvUeLiHbOvBsgwNlEf[qXw&city=xAmbhYD^i}]HgAskVoX~|&zip=xw4HE&terms=}H_',
 '/order?item=tshirrts&name=fbP{wjcJdY}_{zTziLnfLGnnXH]_h&email=|LlGcV_gTOfzw&city=OgtmYN[BKEB_tPWZhvK&zip=L92eb&terms=~__',
 '/?item=lockset&name=[F}YCSsksjLOAOGk|hyXk~C[{dB]z&email=ZRpwPe[L`nlgq_g}Q&city=i]]MKVVkdTc~TQPTjayWmyR&zip=Nom6P&terms=mbaGe',
 '/order?item=drills&name=XEpTafkfXmw}njN\\MPsIoeYMXRPxD`LvOYQogWlJvWB&email=vqaIy{\\DoL|HP|ZUTKxBEvjThTKFw&city=nn|SywGDWMcuopb&zip=XAFB1&terms=_{Q']




medium_population_1 = ['/order?item=lickseR&name=Nv]^VUHB`]WjPO{SAN|k[gYOcaP]Bt}S|}{NHgHez&email=|dULjBHA]jiiu[`QjUo&city={a^}Ral|qA&zip=Gx6tF&terms=ezB]',
 '/order?item=lickseR&name=UJBZFlPeCJTcYctzq}{^mP_\\zUsxcoSRszJrzSpw\\&email=_f]WnKsnB&city=AslSw|&zip=tXE8W&terms=t}]CQ',
 '/order?item=lockset&name=T]eedREnfN]ju~MzGLplUfptr|NvqJW`E~VL~gwXTlk}fcb^v&email=ueejOE]x~bBhgu[sVuL|mCR_mpLdw&city=Nmx&zip=1azve&terms=`FmX',
 '/order?item=P0ckSet&name=HehiRcff[HBpHMU^h}\\uZmDWuvwXYEEJvklCr|dCy_qkAm&email=^WfO_s]RMkNOXnSEM~&city=TzLYRJO\\}S^Ny&zip=l9xGt&terms=iVF',
 '/order?item=tshirt&name=kCeS~KROqPOBjHa&email=BUjQXOto&city=gKgL\\[IQ&zip=uBal1&terms=d^k',
 '/order?item=_uafJe&name=跳\uf2f9콹⏽뢂\uf376潄\ue843휞蚱㖰갏冡줳啬܂髯魜&email=庆헑ꠘ뮱炕뭓ᛑꔀ♖㷌氲ˉ澘릟\u082e㠊䡡缲ᥔ釺&city=KX|{Lh&zip=exkh3G9q&terms=FRl',
 '/order?item=WAsyfae\\&name=禃摪\ue25c\uefc4㶂閴㹪ᐤ嶂乴鲟暛\uda85㏴\uf063螤韼&email=鄻ꍥ콸ﳂγ뮻혷☂&city=sEkgFIXt{aLdmP[GwFd&zip=lJ8MDpln&terms=p\\HzT',
 '/order?item=FaDKt|KVb&name=勇觾\uf1e1㈭揧\uec9cƟ䑼ꄆ\u0a3bꓺ簀鑖\ufbcd띾㢑鮇Ꞡ㾘℣娃ㅟ缛鶃ᎈᅪ⋃鰅踱\ud98f⋎迩\uf4ee색Ᏹិ텶\ue049≺哝&email=궉Ⱆ赯ꊔ娴㉁礞甚吵ᝀႨ\udcb1㬅䫒첼֞\u1759啤≌ជꓰ&city=zX~b~ZY`_G{&zip=O41ZXboQ&terms=Z}Z',
 '/order?item=dV{]_oF&name=㑶䨒큼ꆱ흯\u1b7f\udf59\ue501\ue299䟸㥆г✍述鈯糥䑹㹆ന\udc5c䚉幹咀貓쏒뇈d鴊닭啡䓋ᒓ鯀汹粡થ◪沯\uf20a夣궯噌鋔遱셒咲&email=렐\uf544姑꺏\ue465삈젾淘汩ば&city=pNhs&zip=w52yAb8J&terms=PYv',
 '/order?item=Z`t&name=汱ꚼ\udf98➤\uf73d\udf4c涏慜\ud818&email=ꮀ칎抪ꨈ汿庡단ḱ\udf58鎏Ｘ\udb88涊ᬌ蠎\uf74e钰嵲&city=uYRsU{s_YpTfuikm&zip=4ct0XYyx&terms=fdmc']

medium_population_2 = ['/order?item=tshirt&name=aB_WY]RyymZEXsCskHnDlWpQxqk&email=HDEH|fW~cud&city=vavAa}x[&zip=tP4Zi&terms=FSBIU',
 '/order?item=P0ckSet&name={Xkhgy|rfYp&email=zS[JlgUNq~Gk&city=p&zip=wBDpn&terms=nATbo',
 '/order?item=dr5ill&name=CgrmENev}O&email=NUvw|TEwEipTQUbSIQobidUPMj&city=NIuh{sK|tH&zip=txbEI&terms=The^',
 '/order?item=Tshirt&name=SkyHWqVQTxxQ[OFeP_ELyjf&email=L\\Jz{Nyrik&city=sCyw&zip=QyN3S&terms={{UR|',
 '/order?item=lockSet&name=ru~B_iUjL_`aUbW^Jm]bms&email=Rb]veODjSKdTV}W_Xf&city=UG`OUDUPywD^OIxILti~`Lq&zip=Ywpiu&terms=}Qso',
 '/order?item=mjWaUej&name=㇍쀳尴쵓\udda2䯆ᬶ韸∝陂\udc01鼂㡖㗉毟ꋓ꙾\uf0c4䴈읶숤\uf1c8&email=\ue7cf\uda22ⴘ煾ၯ쾉&city=_\\JK`EPzz[ICpQhEtZcNYF&zip=GVRVYQm7&terms=rh^Z',
 '/order?item=CSKv&name=\uf763㕍\ude11ச捺䩜嵨⻇\ud895\uf5c8롶䰧ﹽ츱䚫쩄讍ᔰ\ueef3\uea8e饟\udef7\uf2ad鞣⪤밯姸&email=揸鮟銁쿱碅ᢉ묺냸ꎏ쳀胜灼膅뢝ȝ읺\u1ada㊏鹹ꈾ騪ￎ쯏㟫㥒ੌ墑垊메&city=tjQdDYp_qO^kOM&zip=UgDfW6u0&terms=jQt',
 '/order?item=E\\xKsWCS&name=ᯝ錫烘쯖钷캃\ud987\uda0d誧徫ﯯ墪ᔑ淂抇섴﮵垷Ϊ껞敫䖦䣣燬굌佖\udbc1㎈쾂﮵骠䗌\uf440禭\ue085疻ꩈ鍫웝鯽\ud94a摘\ue070&email=ﳝ帼斦ᑳ朚˄漟⑿⋩\ufb1b矷&city=gAX}sVsxnYB\\LQsE~TP&zip=reiWkahv&terms=ERns',
 '/order?item=qtV&name=\ue3c6沏莄犢倕侐ퟏ㶣ຂᣄ嗺툩꽚㨹ۮ\uf42e뵻\uda62\ue71e줖&email=籟쐖鷋崹줿墢罔쉈팑퇨垬塀曖Ⲅ䘅뒧끘\udf0cថ䖘ᑑ&city=osQjg]jk[yU&zip=pIKp7EjL&terms=PfP|',
 '/order?item=NGIMl~sT_&name=嵠㹑Ά䥢ᴢ茥抭썡叜\ue781\ud863캿紑냎컪㔾죫戨⢻㖻Ⲓ倳牢ﰰ⡒\uf579Ⴉ桺\udc65狭ᧁ갖渜ᮋ囓ꜭ\ue0ce緄﴾璈艷\uebd1浄昳퀴䨀⟹&email=ꃾ갾뫉斿\uf857몵\ue34b\ued0d㾳撦堖\ude85㈳\u173c怏&city=~pBZRtWQvLKChMuxAa&zip=VwhpHD0G&terms=KwoH']

medium_population_3 = ['/order?item=7rill&name=oG[[L&email=WZHJRJMM\\eZ|xOdmm&city=IKYBeKcFxjbjWO^t\\kxQHz&zip=OEoHC&terms=LDu',
 '/order?item=lickseR&name=fy`\\WZobiN~gEvl{U&email=OGzRU|wAvs]FG[Ee}LxmwWHj`C\\&city=oEEjA^OflQxos&zip=zcaur&terms=KNJb',
 '/order?item=lickseR&name=HXKl`yh&email=l]{H_A|jXVzJ|up]&city=V|IVmZ}cFi&zip=vbRQy&terms=JRuhM',
 '/order?item=lockseR&name=tir[f`hrfhNIre&email=]pdzL^yPYVSxJNcg}V&city=xZAf{JmF~[kRJYvlRH&zip=IkRvF&terms=[xaa',
 '/order?item=PLockset&name=^T`AymfTR&email=fQ`COORt_ua^&city=dNRzl\\gnZFD&zip=zHuhB&terms=V~q`',
 '/order?item=SFU&name=呯९엯뢡뽡Ǚ鼉&email=윴\ua7c7뮫뽆ᦒ剆팞障︥ᓧ㖬\u0ff6W깈Ը\u1758촌⡳튽퀱\ued74㎳吥⳿ᭈ\u0ea0\uf09f&city=Vc_Z{\\DWvv`z&zip=6s1nQlbE&terms=oKFPu',
 '/order?item=JNNCbu_&name=箒ᔮ콩\ue2f7鳧\uf393띻\u0ef0茵繬혏\ue07f羭囝偠䄆吜럎近&email=ᢊ⍉戽㌸勶㶧봹䌹쐇ꋲ\ueb43阉蒧硴\uf2d7샫쪿Ɦ鐘巫㉁\uec60뇨獪獯₤맲猝&city=jKliVjtAzl\\^K^wITrbqQ&zip=uRHWxacu&terms=aKq',
 '/order?item=u`[Iq\\yL&name=\uf07b얢\uf80f\uee8a\uf6e8\uda99ꦨ\uf5c6熓\uf536㟂䜧筒㢙&email=ޔ쟞賤懯쏜萏コ\ue09eꃊ鉩ⅅ싉䥇\u2cf8\uf83e䁱ẛ螰ӹ柘\udda7&city=jbuijouf`eXdt|eiRDiuCARf&zip=7x9gnnq5&terms=C~J',
 '/order?item=OiJvac]{z&name=슋ᖧ㹕藶̀⠨狈&email=뚰\ua7d3䞻瞹㹊闋잩&city=GTZwC|\\&zip=wnD2RnoB&terms=vdoX',
 '/order?item=^JBTNtap&name=♏\ue9a0\ue87e혯囐꺂罱娞鿕\uf734箁컓❑釚罂跣Ԥ⑺黋涸睵̉匌脌詣컳퀧㒋؆᷄ͭ䁳愹큯쁪㼗ၽ&email=ന垁販\uec99\udc2d౩淝៝⊾圑頣쟈（弾뺣ﬆ\ue998汣\udd45輔ይꖒ&city=MaZhkR[QR_&zip=YDNXbBTW&terms=I^YYG']

medium_population_4 = ['/order?item=Drill&name=ZCmOvTtitkkEac]Ro[Fnu|qqYDcznL&email=Lt]Z{^OjQinuXxU^lMAU[&city=qDuMpz_TlzkZqfEZ&zip=ytz6q&terms=|YVw',
 '/?item=Drill&name=MbClV|ctOCq&email=k`mY_}&city=yOoX^rz`GUxSjufolrOVbm&zip=SVqIy&terms=YtL',
 '/?item=dr5ill&name=srWhCPAWQPR[LbssRFRBlRtIeoKo&email=|NC^CG]QJY[fsWPfh`\\BxGIcv&city=RMXCru&zip=wBYrK&terms=Jnq]Z',
 '/?item=lockset&name=JeYTjG`XWXLPwTWJdcIHLdRun`ToNAiT`YIGZa`zNqXjR&email=bwUxAHfuTq`bmh&city=grQMZOjadHV}oZj&zip=yG56x&terms=RJ^z',
 '/?item=drills&name=]VYnf&email=pAoQRjgX{uaoyHJyTESrhQ`&city=OsW\\FeDtWz`uJVqLJ&zip=zIn13&terms=gMLO',
 '/order?item=tvLnc&name=㑬荦돡\uda99굍鯲↫뙫훓곈䥤귷䎻楣է堳䑽릤샑㈇\ue755\uf093둞\uf268⎳콡\ude68᮰鶾씉\ud976\udd12쐡Ḙ俆\ue7b4迖&email=ꌙㆍ亮ᙸ\ue4cb\uf258\uefc3駅&city=}z]HKR&zip=b7jD5nTw&terms=^ugNO',
 '/order?item={Hxp&name=缱⃜魵\uf731졃翳Ꙃ靖雀袂贮켂\uef02\udaa0䎓㙓䞩ꃳ糹緕鼖\ue887쪸納훴싙␥찲䘹奞糽ⱆ薃䉘䚡&email=㖝퍺忡씤겈寕\u9ff6礸ꨔ덬힏&city=FqMH&zip=W17uf7PH&terms=cex',
 '/order?item=S^uvYog&name=闋臎\ude91箷ꉲ폎㕞㙪\uf5b6㯣ኢᤕ붿켐ꏨ\uf869㶤㠴爔㍕\ufd47瘸ᐻ佑褠宽\ue77d稦倍ꕢ㲠\ude2b侠\uf73f䆰峃⦕ꔂ䵪滼ァ炵㕜빥&email=鷞Ⰱ癁灗᮪놆칈&city=RArKQALEiq}VXkhJqU}nFN&zip=y9PjT06R&terms=pjXX',
 '/order?item=Nuo&name=ζ뎹灒㷄樔\ue4a2탚樓⠺뽞\udefc橉⮑श쾠ऐ\uefc9⩈䂧\ud823衐稏ꋽ콒昿\uf223䃛宬죴\ue142褅Ỽ둥䦞㐱桹嗖켰䡚鄞爯迯榁ﳅ茛蜮&email=尉窋᠓놌ൿ㢱㡙놃퐻퍱\udbed\uf617ꇬ둎⽴ꖿᎯ\ue892鯉㰜遭\udb4a迏䥐璋ゆ쀡汒䑘&city=~C&zip=uQ8GPioq&terms=VvBfQ',
 '/order?item=Bmfe}eEu&name=쎆㹽ሱ寿ꪢ➻쯧撀미㇁㹿&email=籙ꝙ\u1942飿냔죵ڗ\uf105횷趹濍杳㤔葘֗鍰\uee9b\ue590&city=hPkI&zip=GJrqmVxx&terms=jR]S']

medium_population_5 = ['Qgn&email=~ZiPMHb[GEz|Ni}NoUXvvuA&city=aRl&zip=BV8lA&terms=cqMI',
 '/order?item=Drill&name=MMeYHk\\FFbMSdtndL&email=wgjkeU_SttXWn]ljX&city=BJrKP`IR\\禭zip=ZXYrW&terms=m|r[',
 '/order?item=lickseR&name=bS_cgaPBg[q_t{mKj~\\Zp~F&email=vBnzh&city=TBFGhxJq]WYsPS&zip=W8GfM&terms=|hzpX',
 '/?item=lockset&name=}mp[OcP\\qbVJt~I~PRbbQqlv|K&email=sy}oq]aEd|}NukUB`IIg}XYn&city=|ir[^O&zip=JVjOG&terms=YlGd`',
 '/order?item=lickseR&name=|giDSaCk|sYmWSjHLjnRp&email=Dpro{okHqE|wFLU}geAmnP|sXi{&city={\\tooFFgaHMG&zip=cwwgL&terms=I_XZ',
 '/order?item=`oPG&name=\uf0c7缧쑁祫궣ꈽ멟\uf389杭Ↄ뀆㧤\udf8b띜䔛&email=䠧오働ᡗ௴ꉱ&city=Rq_FcrLF}JcLcJsKpQST~a&zip=NLyhFZ6a&terms=_WkQ',
 '/order?item=wFZg^]d]z&name=힄邕\ue6a0氿ᦉ汘ￖ屮㒔啩檶\uda18᷾谍瘂㰊&email=Ɫ췣摣恐ʺ犣\udd96୰穟⿳褿먖苹̃罷\uf526勲앯\uf04eꌶ驞뙣ੰᶤ⡜&city=JF_AOudckPxTLiKPintEdMfU&zip=Oc0lXCcV&terms=DAwvT',
 '/order?item=RKccDK&name=嫲瘪貼햾趒䩴뼍恔ɠꇐ\u2d2a⃐䷭担⼬촎瑿∢꿹䃡簈荋㺄࠙˝\uefc1뤏&email=\uebac鰃썿걤髼Ⴭ\u1755펬ô\u1c9d랅㵱䰵恾㏺疬붵\x9b\udb2a䣧鐀端䱥\ue943᳖&city=YvbnadWX_&zip=inXdnuTv&terms=w]bB[',
 '/order?item=gORg&name=朘뫫ꔻ˵\u2065떥K싫ɸ蕪⮱讘⏗厀狼胮⣗錣&email=ԃ䪂㻭Ģ篠\ue0bcᷪ\u0b34\ud946맻鐤鎖켱緽秇좉\ufddb\ue1bc鬕\ue557⺖뢛蔊㱖齲愪&city=DCofKvy\\IxTqoyN[[C]sjB&zip=q7tfVgik&terms=Qo[',
 '/order?item=XM_Q_N&name=혞骵▇ᅇ≛ᮝᬔ\ue4cc&email=\uf30dㅥ鶙뜸\ud9e3\ue5a1舘\u0dcc纤㹠\u200f&city=XPUkQAEnq&zip=buHfdaON&terms=nQ|d']


hard_population_1 = ['/order?item=t[PHhU&name=퉒忳\uf163ꕆপǬ懰鱯ῳ㼩閗♫\ue655靰\ueb12荦痿ⱨ䁥蠎쑿䬭㉍휥视麍봘ꡦ&email=鮉딫䳶䳟َ䡠㵶痏諭⛁ﱱᠲ上遶㣜&city=AGido|kXenKaWhrri|~EI^&zip=BCBTtm2V&terms=ZLL\\N',
 '/order?item=rCVk]jFH&name=銽畹\ue84c♖㒂\uf4d3健ᰱ\uebf5쀥\uee18\ue571ؖ諹ꦏ흇൨\ue84a玞縆䚃שּׂ\ue1d8락곝ڗ㗉羗ො쮟\udf8fᇂ땈\ue754稛⊂&email=⺺ᶇ鞺㑸㏋\uedc6辂螰\ue065볱㟢Ⲁ\uefcb蛘⟭Ⱡ\uea8f竚ୣᦈ艒⦊&city=bskbcN&zip=Tl7f5FOy&terms=KqKw',
 '/order?item=]lh`ht|jD&name=槶뵎\uf58c눉\uf634\ue384\uf669﹤\ue7e4껝榏䏿\uf35f엒ꯐ槨㐁ᬅ뒃༞䦚㔯뼌鄁蝜퓙&email=釟̀꺀佳ꛜꋀ䪢\ue4a8䫠૬稜櫳⠡\ue053㠗ᢥ褺&city=BvjGLXk&zip=5v3x6H15&terms=vn_e',
 '/order?item=`{UqMpPm{&name=㦶䓏ᬔ䌰ꕺ蒝䋱띫༩邧\ue8d8Ḉ慷右ઙ㝵炱痯곆㾙奞邁槾ᕚ嚺秙ዔ泫찛ﺺ䈯瑎▼㎢鳮&email=痺\udb4f헞ᐆ舊\uef20䰃졹놟\uebe5沈\uef44䈖储◍归繌Ꞝ攭ꋳ儡橴\ud914㒒웪玾벩긦&city=oWBze[qrXK~|G&zip=cL26Iq0S&terms=BnM',
 '/order?item=eTCCj`XiO&name=\uf381䜜ᄊ\ue8b5섕䄲䷲灗䰉聪䧢戤\uea30繅ꠞ\ue53dༀ雅\uaa4e材뵇㍥\uf58f\ue9e2颯ꍻ\ua6f9殄禭厨\ue634嬥标ṥ幍訹ꖠ\u0db2ꃄொላ뷅\ue18dპⳡ&email=ꅛ쓯묷喥葇蜿뱼앪ື堅ᘎ⒊&city=snz~XvtVe^HVymPhBx&zip=iZjtUVxy&terms={zrW',
 '/order?item=\u0d11\u05fb&name=ꩌ吀ࡍ麯ေ\uf0e4ƶᑞ\ue1b8&email=垗㐸\u9ff1\ufddb㻤&city=끱勀鱖俢ꓖ\u07b2⊑&zip=8VcSQ>TUw]&terms=㏄\uf2fd',
 '/order?item=棔㱘ⱨ&name=๎坢⼫៴焍⭃졫\ua4ce渆\ue4aa\ud971鳝闇饗詩ܯ꿕梥\ue1cd꼪\ue120\ue502櫓퇐ﶢꆐ㣺\ud9e8\udb52낟솆됻Ċ䐻ꤵ鐵胄㡿ນ鄘ᅺ\uf7f6휎햘턍॰칟፞㕹&email=ꢾ창\x1bⰨ쏣噜⣇\uf2c0蟯⒭욆냠ㄜ䪱᠇꒧̺⣌\ud803ᐳ畆ꪹ臸밐磀Ɦ瓃&city=䱙墨ሜ뀸権渧瓫웏㫿ਫ਼&zip=s4sEc@h{\\0&terms=㌟',
 '/order?item=粃䐁&name=ꆈ띣駣\uee82儰ᛧ肣\ue6abꢅ⁶匍然Ꚙ瓫屼⎋ᎆ겟䇯▭ꕜ輻ᴄ\ueac6ㆱᔳ葙Ｊ戗輠፯\u0ae4衙얭歼\ue714쎮껦\uec0e\uea74&email=\uda48\ufadc釥뼫鵝ꖽ\ud91b⻦\u1fd5託㠎鬰\u0e98㝞蘲&city=쓞♊℻锐佲&zip=u_}tqHc7R(&terms=\uebb2猬\ue545',
 '/order?item=藿褚\ude99\ude840ᛎ仸겔&name=\ud86b㷐Ģ‱礡탱\udc4e伫\uea92Ⳡ繗䥏ꎫ柆ჾつ雁弮篊&email=鍀⺼逕䉳绪뤿\ue2c0虥洘觅賚㝤Ǉ࡙н县졩\ue07bප硢橼迼专寵稳ᓸ鷤儍\uf593&city=ݵ講蜈㴌\ue745\uee91\uf7d1&zip=L>bbG2r9w5&terms=\uaac6턧동',
 '/order?item=\uf0af뒈&name=껪\uf04d坡㐫⒤䮀ԡ曯ｒ贶\uf02c嚔㌞壠ᮁ같屧俪殭䜊韜\udf41尪嗽&email=\uf5d6鲌☬\uf25d꣘翄\ue9b8ﯽծ建\ueac7紙埣빟꿎\x8b\uaac8ࠜ㝥蹞夳漢\udd23&city=剓\uf170銑\ueeb8⪥湅\u2d71&zip=0]il?aL2KC&terms=읔㈤']

hard_population_2 = ['/order?item=sDw[brJ~&name=⢦甏虾輔\ua8cc鏷᥉㦐䷮䟦嵫뛤爕ꎑ㕘僸Ꙕ㩽᠄銿ﯻ쥜&email=䉷恴뺘㎪氼Ꙕ\ue4cf쯖Ⓙ齌\ue2b0\u242b\udafa̪䟗&city=GHa{HVmkFTEPUoSF&zip=rrhnx1hI&terms=cNGCQ',
 '/order?item=][wrIbW^&name=昆ꢁ麌\u205f෫썼얳슢䔊&email=べ髗宊ཤኹ䢞꜑ፘ犫ﻄ氼หシ퓓㑝䬪\u1ccf묖哾&city=`pUBH&zip=929mzmfM&terms=^}iea',
 '/order?item=^qXGVtWU&name=䬀諷\uea31\u0d3c；&email=ཉꔺ欽㎉迨㟻䮕ᤄ&city=XysF[&zip=pRY9DC7r&terms=Bjzdi',
 '/order?item=ELXcZPR&name=硗㍄狞歞롬\uef9e๒ⲙᦛ쪓\uf7e3ິో䲖绔샃젪ᖑ膡쟹\uf585\ue254⫕춦쮻瘭뷤\u086d蚉叏땟\ue0bb띴㓟娳≀\ud85a덑緟\uf4b2曆삷&email=鮧逾載䟨탪펪쭜棱䏬쉈腈༗⢶顼\u1b7e楬ᤣ\udbff\ud9f0쥇Ἓ傀\ue59e瑚鑣&city=WB}`jgtWqJG[nX&zip=S1HcPBho&terms=G`l~',
 '/order?item=vw`&name=袄ԉ딄੧␥\ue192붥됸ﯤ쒹⑉滫\ue62e⏑\ue2f2\udf59籱ﻙ芦&email=➒\uf7c5糈ڱ麫⣓\ue2f8꺥鬙\uedfa毜핑៕\ue0c0⊺ꢷ\u1caa㴓ቶ㰸쩋홮埌Ɯ돺\ud807&city=iQkCp}yXU|bjyYe~_S~Y&zip=7yvWCxyv&terms=BDk',
 '/order?item=ٯ綮眏餣雘ꢏ&name=㒆\udcd4莤붪ஆ匛癫䃝\uf221깬\uf379\u1878Ꞇ薆\ufbc3萳㑬\uf418嵚購\ueadc菘梲䫺넫쯠忇력\udcf9楣\uf6beˮ쾡\uf1be⤯곟ｎ\ud89d䣒&email=䟟檖\u088cﴹ\uec80\uf340讪\u088b䝰鸯䜏ꗟᦫ斸㥡㐵钒챩뚉聻\uef3c狛鎋䳾╈얺證\uf47f&city=၎\uf3d7ઞ㱡걡\ue750❞\ue44b㻳쪂婔縓༶릍좐沜ṏ㾙&zip=)c#TSMsq=7&terms=',
 '/order?item=⊈\uf0e1ꧢ煚攻쌻춊㹣ꯠ&name=䭹偽ԧ&email=浝ﺴ홭䢀ᩮ\uf6bd桅\uf3f4&city=\ue8db\ued3a\ue921㫗懦춺鷳\ue98d낢᙮蛗\ue8c8\udef8簕䵢\ue107跖炒&zip=]Wn~XrrFp0&terms=뒱펡Ƙ묂',
 '/order?item=駠⿳袲&name=벟\u2bd7潉隌\ue0ae\ueb17់凷橵ᵩ뛼뭛댴催\ue452챡\udef8짉\ufbcc얨ⴸঢ䍮穚造\uf441㦖륬ዯ䁏콲缐♶ꠓ㞤춶宎\ue0a5錓訶螈&email=慥&city=䝕뮫丩\udbc4凷✭讉ﷁ鎛꒶檕醟\uddc0평柼䒾羱\ue42b\u1757&zip=}U5[l~gtD_&terms=㼿Ὡۄ⇀㾦',
 "/order?item=❉\ue7a6껈忛&name=郁齉\uf678阣灤裑몂據櫂롪낗삭랅፼ⵤㅑឲ끽鶉㪰秌耙莜꛱䤀暾ໂ\u1aeeﰡ䋉&email=뮜ᄽ뤨Ꙫ쿖硺\ue5eb풠煬\u19ac&city=뿛ﵕ\uf239ᵝ\uf181鋹\uda99臸⠍ဩ\u2435跄繦&zip=2'3nNDfsB(&terms=榓ﹶ豚콇႐",
 '/order?item=⢈\uf88e翹ꦓ겾燂鉐&name=\uda33ﶁ\ue313꾊ᨼ祗ᦪ좺\ue9fc붥厑蠴䡙䆃闉⾯鷨횞蒫䜔繀궪幍툁ۭ\uf008ᠲ\ue546ᵭ싎&email=濩祹튨\udd4c局촲ꐊ䶔旎ꄈ鱌ᦽ纘퓣녲얧ᅾ㭆磸썟咃㖙綪\ue29a祇ꔖ攀託&city=紋䯕⢶&zip=h4nt!f)B~U&terms=\ue7a4ۘ⥬\uf80a\ued73']

hard_population_3 = ['/?item=VbqKp&name=ꕂ\uf637悀\uf0f5Ꮄ䅡⪅䘱ᬽࣖ⮬컚撒ﰥᙚ䰪꿽嵠⫼脔굥\ue5f7\ued71㎵ꤽ偢湲펍㋭侁ﱊ碾刜괤乪麦쑑ﮟꬨ뻜擺\ued46&email=\uf7ff㈏ꈸ妁\udcb6뾘嗂ᢴథ⼚㨲\uf051ᕃꇔ䰁\uf4df\uf428项緗\uda27Ｋ쭚\u10c6햚䅔짲&city=Oq^a]fq^fC_B~noNsLWDTG&zip=3zejH7DR&terms=SaSfs',
 '/order?ite',
 'VTsPPK&terms=~Vv',
 '/?item=r`Y&name=稼㕑妑ⷮ㑫ᯱ徫貏뱓⍼᷻뜂湌䩙蜤ﵞ鬑殽떚\udd47ꅌ褭뤓㈎￭擈\uf42b垍奎᪣霮葁邟\uf870㴽ᱞ䞇ᙒ룵&email=ﻗʋ⸃畳齻ꢟ겪뱎䭀ꊯ͋⪼ꈺﯙ媒࿐\uf7d5㿱陔崀抦ు枚絭⛻쉐㨫兙&city=qBWWy^&zip=2jZyOYHH&terms=OaG',
 '/?item=ffBUq&name=洸\uef92騇ﲛ\ue660鳥췙ὦፚ덵坑樀浽檪枧濹鿕✚쑲ꐷ⽫塵䚌콁\uf493\uedffⲊ鎁숲☠㉤孝㓦翆Ļ\udb17헿\uebcd켙踑鑪ﴣி\uea06畨騺쨾⪅&email=ೲㆰᡎ⌘┏揳ê\ue86aⴝ벞騎ⲹ⩍괜ᡄ骿郆䤪亀㣛뭾ʷꔿ\ueb27眪\ue5deȣ&city=D}z}Kd&zip=2ENrfDgK&terms=NWP',
 '/?item=葹&name=ຊ⃒肪矿䜔&email=&city=煪\uecbeꌿ\ue96c峃䀱챭佥뮨雝萢\uf704&zip=F/u`?!LUFn&terms=㞑',
 '/?item=텊ᕘ諞펝懆&name=唗춙爩Ḻ⤔Ⱉ崓溳驞ﴸ⸎㭑숱ቱ쬜㔿\uf068幖뷤\ue2a1&email=\ua7bcᖏ\u1df7ꢜ煤ⶰ觧旭羯\ue45f\u2b97昩\u2e62䉳\ue141焏&city=\udbc3⤅跈즹ꕳ&zip=2lY#yEq}<K&terms=',
 '/?item=&name=晈뵳孏鴓ꄬ놾\ue5db莚곆薳聘袀ᮮ즜쵤\ud803\ud910鹈␟憂䣣\uf122된㲟Ę劬鄕憡筲펖\u1ff0쌖뻌㵐\udd28ⴒ扏䪌킼磎ꃎ\ue7b3&email=糷쬗癧&city=鰱䶑䦟̨煔錒椿\uec8cﯹK\uf4fa覟晁\uf526䃈廓Ꜳ\u05f7躲该斋\ued38๒\u31e7&zip=N\\d\\z*;dd*&terms=',
 '=㖉ꩬ콎\u3102꺤噔٤ᚕ\ude5fꭒ쨩\udb26捿\ue585ꍚ鲞ﭱ⊩&email=㥩\ue4bbέ汫㠝ﶓ｠\uf20d䫢ퟧ松ꩰ騎坨ò퍡&city=츉⥰褜衚\uef30&zip=1\\}UX4M7Bq&terms=酒䃌忠缕珅/order?item=\u1f17峔&nam',
 "/?item=刟쟍ꖊ&name=椽\udaa0䝻䘅濛\ueb96\udd63&email=턂ꜽ䩽࣡⋇熕蔳\uf1ce\ue468ﳏꢠῑၛ焱Ḟ\ue2e9짤矘祺䥪쏨槀\ue144㜑Ļ뛕\ue3ed\ud85e鋉&city=빅笵녥\uda95䯜鬴\uf127뙈䃫왛&zip='Y6nE`-r:h&terms=튮"]

hard_population_4 = ['/order?item={GQAo\\&name=︖䖵Ᏹ崶냬\u18f8ʤ븢봏抭ﴱ덽备됳衝\ue475ꎵ鹟ᢍ饑㈩㨭蟩&email=ኑ\ued7b\ue106澦\uf71c墜빈钝鹾ꈱ윌㳜鱏&city=mlYOGMICD&zip=cDwYaMPA&terms=cAhY',
 '/order?item=U|vR&name=镍䞡⼪웟掕ຯ郍\uf5a0튦鵘馢츓䗭잉띙ऌ좏㠪駓ｇᕜ&email=㳳瞧毎㹔薝毵屠莯끋&city=D~TeCYf~QFNKwO&zip=gL8X1QbF&terms=MVgj',
 '/?item=xuclBXiyZ&name=흶脨㚩ﾙ皔搹⍦ㆉ&email=㗭ꛋ쿎ຂ牂陣촛犘諧&city=FV{KYyUGn~WilV[\\cG&zip=3IMQRRFj&terms=Vtho',
 '/order?item=PiCSCp[m|&name=\uab1a潄ぎꃈ챉㳒篬飣顮㞸⬱\udba5ஓ클艘惇⥿Ⴇ씿띱꿉妘妏ж\ue764떻놽ꄢ걀\udf4e\uf3b4ὗ啰빌ꧣ趦⺇졚桐&email=鳏嚑\udf2cዠ挾輢덃苤얊糧፩︀\u31e9ꨆ\uffd9퍇勔炭换\uabfa\uebcc挍㉽&city=DSw]&zip=ppKffx4i&terms=`^s',
 '/order?item=fYwHO&name=ퟞ뙑︂ᾔ㯣魟\ud873䀓ແ鸨ɒ咍鳭춥ꈟ푹㴂븉熇裴⅌옉\uda87囋♤됽嗵⭭⒄儥䢍&email=┎輕Ȟ\uf8e1岹ꁘ籱鮜瀧뗷ꥆᵠﰕ\uef6a튡ᵱ긻샲㑥&city=^ag&zip=s1YGTgmH&terms=zNuG',
 '/?item=ꮨ츩힊&name=滞\u2dd7麟곆\ue4c1ꄎ桭퇊綔\uf765\ued54㏺䉌郹韩\u19ca聏筇\uf6dc䙋\udbbd哧낤࠸&email=ᯧ&city=蠲긶ﳩ䢙⿺\ue2a5\uf571\udfbaง皖⼆㬎䯰ᬉⳬ&zip=\\OfL;F13fi&terms=ｒ䍝立房',
 '\uee5bᦜᛚ௭&email=㵙ȹ蓏缀黨㮼\uf117ꌰ騤혒\u20f1䗇㑝\ue1bf㰞&city=ꓧΩ÷⊰朖⺆Ｚ\ude56倯袀\ue286鉴ꂾ다㠉&zip=O&]ZkWv5X]&terms=ⶩ➯뫞/order?item=㴮ꜯㄎ끋\uf6b9&name=惟ﻶᎇ㹄㓡帮扆\u09db',
 '/order?item=⎠∑귁\u0bdc魔鷋&name=㺶垭⽫ﵵ෫뵽쒊व괥뫊켠霭翳≅\uf1d8튥恄褩髑\ue855臒ẻ萫퇇耮踘\uf755\uf06e焝ᚤଓ䒜鈌\uf18e䶦슞巒闁붩땿蜻똫&email=m駽\uf29e䚏ꯨ㫚큫',
 '/?item=\ue46d㥨\u0bd9⛏疯&name=\ueeee첚嚽䫛聘霧\uaa4e텄푦♔錚ꃫ\uec34\u1af8ਣ뿘\u2009금鳁孩ߠ❑饢褏糹롳뀩잫壄㬐\udc17꾛\u3000ⱒ쬂拽肂鹢拖䮷갎&email=₂芣暱櫜厶ꒃ쪏Ձ㓃秦땎྄递Ƹ䀴⭇觱폝녭灂ഷ\uf2af桃\ued6b腔要ೋ&city=\ud80a\uf5cc廇\u0a0b䎘뽉\ufff8\ued79ໜ&zip=v^E$eu[TB^&terms=줐',
 '䘟辁ஷ\ue411葯\uf47d&city=띡疥該蔊㹛夓Ł稄扟똆䆓甩嫲ꕶ漈&zip=W=WfM:,~t?&terms=\ue926\ue086忿/order?item=\u0a37嶮떇蜑&name=呤ㆫ쒉둿㓆人ꩳ힁ྫ\ue6bb뱶祩ࣟ틉싊⪗斔鶥䤭᮫嵙℃壪ꡟ\ue77d刍ᗘ\ue80d戕䟟㾸ᡚ㉣ퟧ&email=镁瘫ʡ\uf7a1阑垤']

hard_population_5 = ['/order?item=ywz[uqm&name=ᙩ鲭㳩곟侃깭ᕏ캪\uf60b\udb4c쵶↘态耷蓍庤⤺ﾐ䐫脋ꛆ䙈֬෩ڭ挪躡杶％๋폙뀹&email=햲\uda2f䓔⠘㋜굔꼨Ǡ슽㘠甗鑝칝ⴟ㸑\uef73㍽춥퓱徱蓟盱\uf811䊊&city=RwBCJlH{VIK`T[&zip=yME8KqCE&terms=wvkNR',
 '/order?item=trhjname=箎槞쉶㶙ꏚ塀嘶㤲Ŧ쥠\udb98螬℻\ue927宱ኰ旹㒑\u2bda야錴椽샻펹ꖄ쒶email=䰖凕ᦁ踏컗ꤺ䛁\ue387縲ﯬ邓뚍\ue328ﺝꌯ䇆롊ⲅcity=orPP}PPAiKK_EG]AEzip=AnQv4EN5terms=oay]',
 '/?item=aVH~ae&name=랟뒠龰馽竉䨓\udb6e᠔⻁릧尮䋓㵄뼕\uef6b㾙엁玨ﺈ颛찡&email=ꒉ覩찍ꌁ䑷踳츮ꜩ҄不㌵ᤢⅼ\x86垉ભ捵졿鹰讇籩&city=wk{\\eYqsY_tS&zip=Dq05oPfN&terms=`Pn',
 '/order?item=yqBc&name=쎉뫒',
 '/?item=IUJZ&name=롵型ﶶḳ㑉Ỻ\udbd1郝匰ￒ쇑鿔ᢒ烂ᣖ粆귦♠ㇹ箥\ude05⺌⮁樕䗋컴帹쁿笤&email=䬃꜀\uef19숩\u09fc퀟翀豶㓀ﴖ軩쒚䖫㽙ᣬ㍈뤈ⶶ媛\udada⥬桘ꉗ粙&city=mO~ytcr_DlGdmelGWab{&zip=injC09K4&terms=ibhAB',
 '/?item=淛\uf08d\ue1dd冓ܘ&name=쀬鉴ẍ岃뇾Ꙗ㞘귤惚ᢵퟰ\u2fe3⫍潶\uedb7&email=欐็됊㧎櫢蟧₆␏ᑔ빏䊙\uffc8텦戚嫥\uf04d뒇覥⾌ᯚ丘重琖&city=\ueeea댾鞕&zip=("|t63n[)L&terms=錳',
 '/order?item=&nam=ݵ㎾똫藇鞬瀠⼊铟졷氲灜穇뜹癗檬풎뫊䡏毩ዑ稭巺沖괭쁰砺\ua7c8㜧ۨ쮰\u206a䟐憄ꆭܡ俥䡾賈뾹䨵엗ꠅ☷\udf2f陻&email=\uf3a2楓닽&city=꜌\ueebbﵔ墨꒡訂䂉텵랢収ꢳίꜲཐ钊㠍ᖒ\ue3b7䶫쇀ī丠ⷠą&zip=D-937=!pa>&terms=',
 'y=\udb07椃煏茇ধ\ue8f1\ude23跤ᔥĭ網\ud95d₥裖㐊畣싲珆&zip={<XLRmBny3&terms=栴/order?item=㲭䬧&name=&email=嵱嘆릐鳁엃⨌਼힛ꁧ꽡焖㪾人䦅퓲ް揫搄䌤묺蒸䨘ꠧꑚ\ue3c6섢\uef30⸒뫪&ci',
 "/?item=↓&name=\ue745栍\ue11a삧\ueb80\udb1f릢璚㶃䤿솋ⳗﳇ艉啸俣槶᱒\uf20e遛\udbbd肉ꈼ获촾䅴烽꧆ۄ៶럻ꖤ썧\ue576鋭먲ﮨ璋ዙ\ue0b7\uf708&email=澴띏撦\uefe7蕽癰葊\uf099쌧\ue79b枓롐韙㳟㠉訐ﺒ딬虥悄䛱硂汐춲㻟&city=ꨅ࣠Ὁ扶翻솣\uf117掤\ue66e振뗌탽&zip={'[!1S_&3q&terms=",
 "/order?item=ꨘ輞&name=菛뻫䲰箈ꢸ籂\udb80嬶\u0de4祵ᔞ빹ꨖ眘䉎幼袠&email=\uf200塗偹&city=쿃\ufa6eꯠᖞ꜏\u0af3骵닻䛙㖚\ueb71\uf8ea朋䞩ï&zip=b=;-6'.o+0&terms=竖"]


complex_population_1 = ['/order?item=闥&name=\ueefc빤㜘㏵㊃판쎬樐\x10∜蔸頋\uee70៝杇璹갅Ր\ud9ac뿋ʯ\ue33f젤줴⑱맹ꅭꙡヲ㼇䚄쐅ố鱙틹伇裀譯征핡╃Ꞥ橿&email=\udac5阤桳പ䑱촯ꤕ뿓&city=ಖᠾ쀼兲ꃚ&zip=X=([c!;F+-&terms=았基훜',
 '/order?item=쳵桜躼뚤ﺖᆦ&name=\u2efa䴣㒖쭘䬃ဨ├\uda03쳚\ue5d8倫镵\uec0e⤩\uf297勊꘍鶸玪ᨣꮸ\ufde1䬂⎍苛㏃䌟\uf44f锱猩랶镱ꕤ&email=Ģ럢\uf4e0㔏龐\uf42d夨탽⸣\udde4&city=螚\u0af3틩이ه㰄泧苋│⊏\uf729鎷阍\udc84얩筅馁⪸搰²碻쾽阿폊&zip=>AnE/0D_*S&terms=',
 '/order?item=&name=錏㨛\udb14홅俐혘췟핻鷂改し糨ᙅ껹䭌硍绱ωꨣ\ue05c岆䠧享⾭&email=Ɥ㌦랟ⱐ녟쇛扖閙墴깴\uee88\udfb1&city=⯀侒ꯕ⨬\ua7dc뇄㨦칸꜃먯⛡﮹㉴튲&zip=iUr^a}qyKM&terms=晈',
 '/order?item=\ue0f6䓉瞊\uef41\udb91ᱳ漊ḝ&name=뾖\uf64c⌒\uf3fb&email=敻喤ꕮȺ꽞\u173b狄匣綵央&city=雾㶻乯꩞렎┫賊⸵\uf2c6﹂꧟崞ᚣ\udedc錐繗ꄄ䎦턳뛀䇙\uf3cc\uf75bﳖ&zip=2wxhC2-}+U&terms=\uf80f\u2fe1뙖',
 '/order?item=&name=翨钉\u2003찯渪雠鳨⊅폀噶㶚朡ឞ碌응㤸\uea12蔻潬ﰸ䪩\ue9ad㕽㞣玗甏⛣\uef56\ueae2ꤿ㹼ླྀ矮䨶킻쿓\u2bbc\ue880&email=麍㰴\ue4ad浼잂팠佷투浧뮬鬎ﵒ튴罇썴ኣ旁&city=ჟ謬┑栐䃝욹撌킁礪㠭墵̚㱬\u0a7f鶃诓꽠쵾\ue5c3㺟렁歵&zip=u9tCM:up"7&terms=䘃',
 '/order?item=꾢ᐚ遟鸱뱣뇲셮鱋͑&name=䦓꽧ꜛ\ue469䍰\ud9fb룗惥\ue858级ঝ㖜軺伮ꡅ\ue13b訍妰㩤ㆮ墴ᥙ줎ᏽ\uea38\ue061鉵婎隶ૣ뫣&email=想㬒\ue74b涛컺좆\uefa2窟ͱᒽ鞻蠩்쿷읪&city=ㄇ梓짉歞\ud896긄&zip=kmw:C),[Jt&terms=ꝵ﹖奯쎣⟟',
 "/order?item=纋렋ꩈ&name=\ued39㖀\u1771콙屗饒谛ꎜ릘㑮쀤䥪큱ꯟ\u2fe8홳惐\u1c8a䠶Ἦ뮂崗젦撱㱛꺺㘜鳐ᑗ栏釲\ud999恪텨槍軾㵫&email=馊㖦&city=\uf4b1\u3102샬ㄲ淫맄᧫\ue300듲﮾긢\u2bfd\ue9cc⢕斉띞&zip=>]2'5Cws\\u&terms=偑",
 '/order?item=솇Ꙋ쾒薚鋲贕겥ᩑ쯏&name=镑ᘐ긪\ue87eⷦ食\udf4a\udab6뽐\uf4c5䁖\u0e85\ue665᧴\uef81ኑ䗲涩亮⼂腚峥\uda8e䂉蝤ⅲ<ழꍫ仼쁌悂&email=\ueb18ꆓ囯\udf76䊩㵈\u0bbbꗣ٢艨ꖙꛛ粑\uf84dⰍ撫弹\uf032洞㷔&city=蓵狔쯁&zip=5z>`{DD|5_&terms=묰ᖱ',
 '/order?item=槛靍耗ऽ㊥阩耷&name=옅ᰵ\ue509鵨걅\uf62d㷷෪眦돘ㆣ䭍嬮騗˷ം钘&email=ﶪࠔ굡ꋪ筼爻䄗蔯雁垭ࣺ뾿뚍烛&city=ᚽ㈐粰\uf8b4旔创ᆫ\udde9꾋낢鸐迒ᴋꩺ⍙\uab2f兞凭쪴熽崌ၤ&zip=~MWq*0|y+Q&terms=',
 '/order?item=촡뚩櫏&name=塠곊욈鋴ཻ뗨搵䅲Č㤿낾㡮\udbcd\ue787㰗浕哮쁓\uf8f6\udb7b냱&email=咪蔖諍\ued5a媪赌磊뜜婐웰ꗑ㇀槁\ud923癬&city=젩►ธꅈ凈효\udece೭⺕\uf11eᘠ&zip=+2?NqyQ-<t&terms=뗻읊뜠勃窂']

complex_population_2 = ['/?item=䠞\uf25d찫䠾&name=&email=鐶↼핆⋬竡䱿첹찝☘ֱ묾钂ꡫ\uf1d6ဍ\uf55e蓝뫘\ue8b7竣헂ᰠ궊禷Ε◗䲌\ue1fb&city=됖ˆ鑓\ud9d1や䋾컙ƾ冠ᒗ褤&zip=[Yr43gJ@l8&terms=Ꮅ⌀쏘\uf3fd',
 'm=&name=ᄒ㶊冐댱㲨㤃폦෯跗턼橧䆙꺂\ue309䠊\ued1a\uf717뙏뮏쨬爧\uedd4ﳒ\udc6bŇ魔竁\uf561勎糜焁됚ﻄ䎨隊ꄩ\ue555ᰝ⢐ܘ戞\ue20f茪\udc8c\uf6c9オ&email=ሉ늝\udab6뎊鿈栔辕纎鉤ࠏⰫΆ\udd51ㅌॽ\u0cc5ᵼ\udcde&city=&zip=/!u_G_QGL*&terms=츺躶删搲糃',
 '/?item=\ua7f6\ue480㫝糖&name=置醞뙌\udfcb︎\uf773괔䃷䥻ొ稝輺嚐秨\u0fdd檫䓁䂦ቦᇎ攱判ᜲ゠㽉냅К\ue94cꭅ⚶㜲캀怾\ue619ᬒ퐚䜏&email=&city=㘑ꏼⶋ&zip=&(VjYoIFoU&terms=籩',
 '菨\udfb0챫좺眺ꐹ傹\u4dbc挪ẕ祹硠턭\ue323&email=⠷\uf700Տᾫꐑ팡℮梈✖磅뛨㙢&city=嬓&zip=@1f^Pf2p6=&terms=ᰯ\uf707Ὂ',
 '/order?item=㩄&name=䓹ꏴ鯼&email=&city',
 '씛摸∟&name=祶諵隮ㅎ⤶Ų趃⨯\uf66b⟡\ue8de슆\ueb21윧櫜㪞婓ꁜ흰猔鄩禸ྡྷ跱瘝鷘똣ꁥ螝鑱鱨쟌&email=&city=픬撨횻淐\ue418㽃홂\uee70鰨킡澋璛\u0ff5\ue643輦ꎰ䪜艛т趹岀\ue865&zip=4$C<=y/wa^&terms=钰\uf0d0㴝蓱',
 '/?item=\uf71f\u2fee࿀ܫ\ue7eb瑁篩Ԝ쬺&name=豮咆\u197bǁ厊旃†硎㒘ꂢ๏埑焧䦇䖚￬뇩湈ꥋ̙\ue818\uddc3춵욟堭\uda35↱傲ᠰ䪐\uea55餐۩猾둆ǭ嵋∐븤㫂샕Ő\ue061&email=孤䄰啫閂▌횦逃瑝烫刳퍦\ud817\u0b7a儑趭&city=ꥢ盭ꝸ\ue1a1\uf249魑ಹ뭻ꪁꩻ螼\uf539코\ude98俇녹&zip=$Z7+6fy]AF&terms=愬',
 '/?item=줓⺀祌铧하馡＼겈娰&name=ꩲ琿韣㚴쭩&email=춀Ԩ뾍펈萖ᡶ\u08b5樵窌䷈诮⦴鑓죍&city=\udee7㽋睄\uf19b㝟ᔞ&zip=N:;h)vhAOg&terms=욲\ud7a9',
 '/?item=뒴䒡셒彐ᗵ&name=툩鉟\ud990\ue1b1噸ꖆ飽센〇\u206d䗋삀鸃ᜑ\uef0e쾛Ꮌ泷≳孰₪㲰틛\uddb7䍨궑靳\udd14截多\ue10fⅅ琝닻\ue94e鶃뎀㪤ꦈﭔ&email=꽕췴\ufd42藮&city=\uec16鍨\u3130鎺鴞浠쒄&zip=HOT6F~Q/Y:&terms=ⶡ',
 '/?item=\ue395秃刞㊜歋ꓸ蜵툏Ẇ&name=ໃ먉悶怌枓눫痌젩⥅휞뇻\ue7f9岸\u1aeb搯ߓ瞯ꀣ확⸳듕韜뵱Ꞗ\ue482㽯\ue862㺘䫨\u2e7f篹&email=䛶軜㝶ေ쀳潚ញᒴ藡\u2fdb:\ue3b8꼹牶┞쇠&city=滽㖖\uf511컮ݥ弴ὤ㴏&zip=j^%?6u-93~&terms=\ue924ꆭἘ']

complex_population_3 = ['/?item=ⷨ㕁ᢹ៹&name=そ頽ᚘ삈镦뀱鑯慟ꮾ쮳㜪\uf4bf㹑隃䲩钗뱾筗\ueb05ᝈ蛮\ue033洨勎揍䃴\u1cb7猰鈦澤㥥∹㨤뺱툁\uf60b\uee03\u0e60☜ଦ\ue9c0&email=&city=鑟鞤㻣&zip=*M,I2o31?r&terms=矲알핧듞',
 'name=쟕帅&email=⬳멾튅磶퐨쿯Ⲇ쒤嗉퇆畣ꏹ╚震បﹴ㽹䆝&city=꽄鬻娚\uecdd鴳蜩&zip={8>`e[7eF>&terms=/order?item=',
 'ms=҆/order?item=륙&name=[䵣侙⦳摦롄䃦榷靇噎繲腟鎞膾\uf72f柵箍묻㌉୩\uf32b径䊻祉维ڱ㮿殶禃ꬌꑎ瑫ᒽ&email=ȋ燇ἴ囻磮摭︂ヾヰ⢻᪬ꚯ属楴薒&city=㆖ꎝ줵\uec8cȕ\uf83f辧\uf5ee爰\uf579콡稪䅉搏遱쿿娐犚눾&zip=x<L/$nm*Ef&te',
 '=溳憧㽭å',
 '/?item=ﾒ쌀ꕕ确㻃쇋&name=벇搨鉬ꞃ輍歗\uef34᮰叵⎿풍옂ꨱ퓾\udba2饒⋊豊Ͼ紣\uf154푒胸\u0ad9\uf7a7崞\uf31a䡉\uef8c\uef00珓㥓㏿脞&email=\ue061䕨曄㝅쒳愽믵&city=禿쌬&zip=I/QQ.;Ej4U&terms=〼⥰뮤',
 '/order?item=쫩&name=빐ꖶʻ沙ᒾ夜댺삞ᅔ酨唹鲙\udc1f颔ऌ餱鸊ꇉ켻ⓨꥍ\ue29e㖂⚑匚\uf514묲Ϟ淤\u20fa樖銡킫묬ﻙ왻챛☘綠␐\uf449䦗\uf59aµ\ue877&em',
 '⨿괣\ud92b',
 '/?item=ퟨĕὣ眥&name=ボ螡㑩錓旲룸\uf0c7쪮魳쌉鰊蹤ᤧ櫶ቀ惾쏌屮㊸\u0ef2\udd16Ṫ쉍稦䱇\uea55Ꚁ⫧\ue463肋蕠鉰덵毎&email=笧㒡숄㸑\udd4dꥹ節䖂陼坳㡲❆ᬓ⤜&city=樲\ufdc9岸\ue388実ꁁ䆴瓋꓂廷傖埄稷᭙&zip=N]LgDf>m_L&terms=븴⌓눾녤掄',
 'er?item=\u086d悧涖糆&name=簇ﾙ녩Ⲙ텍莽氊\ufdd1䴑ﴍ鎍轰&email=㤲\ue4ce뚰嫍뗨\udf42钺봻䩈䔓㵮\ue511댔閆轎㊸볿嬌\ue372⫨\ue5ca\udff1퐎䓨&city=契ࠬ㒅&zip=TmUO11D,|!&terms=佻뭡',
 '/order?item=&name=谲둔\uea73\udf56㐲鑪\ud8f1\uf826ꈎ퐱팠礬ꨀ\uf217㛲ῧﱒ塆䔿⅃갃ᔡ\ue419']

complex_population_4 = ['/order?item=&name=뵒ₜ共&email=镬苳܃ư柗龻䱆ꉦ䕙䨫寐睤朻ँ뤌┬\u0991&city=緁⮁쨚\u0fea窖襾陉帻筸䩈&zip=',
 'ty=\ue2c9熌ᐷ\uf1fdⵑꨪ샮Њꋌ渉烕훗緾&zip=v_c9%#z6|o&terms=\ue0e3\ue32b쳜️焘/order?item=ά㠴\u1b7d儧踱狫&name=ݬ獙ᩢ洛\ud9e0䤬\ud8c2\ue1ed⤗쎟⃣墾\udacd諻⿷㤚ꀚᶴֿ㱭\ue575蒿ͽ쪒\udad5\ue9bb急돪缑朓ⵜ\ue9eb䯋䢃\ue000䃚\ue95b\udd73늞&email=\ue567㳼꭛㔕䲉䷶⤮闱ꔵⱻ⍂\udf6a&c',
 '/order?item=躢嘙ぃ噭\uf49c佹ᅔ\u0cf9name=㽃卵䘗٤嚆㜆褙簅꺯ʌ\ue6b0땿䌼↭懵Ӄ쨬ᔑ鄚\ufdfe頊삆㪜email=ꆹ\udd7d朣켪㹯갇샹ꁷ嵥\ue91b䞯\ud92b晼ꌽ兄\ue4d1\ue37c횉鮡苃⍺琞䈂䌞垓鄠즲city=\uf3b9\uf876Ѷ䴽틒㎖㥣\ue9d6zip=R;PNwz,v5aterms=ᅩ疻\uf1ac㉄',
 '/order?item=ᷞ⧬㧥옖&name=៴\udf88苂䙧⌏\uef65',
 '/order?item=㘒\ue72dؾname=쬂맴蘃ᰎ셨䛳\udcde눘\ue795∓跋\ue476躲ꐐ摓尉䁜ེ艴엃洆釤ᘏ䌝偧ﯗ彸뱒\ue4d6ུᖀ\ue68eㇿ랝⧔촇牋浪꛶澑玙ᴵ⿁욾ϐ剓暭䖘㪨email=\u2d73줇\ue085夙⬇\u07bc걤㾵ᇻ⮩鶸\u173d箣䌙裘紛뙅☷\uf6bfꬅ㰽\ue0f7夣쪼city=맄鸫zip=4X9>"/\\N,?terms=㞋',
 "/order?item=┈\udd1e&name=習臗\udd2b팴慵變楗퀷ᯢȘ䕔닅뎗䠇\ueec9❨㉪ꩳ偡헢劑뮬涬ꮑ譽\ue5e0\ue2caሌ㠒鉪\uf88dἡ\ue186ﲦ&emaㅔl=⺈\u18ff捯龡巒봁ꎏ\ue591袎\uf1de쿒欣᯾\uf04d莫鈾樓&city=蔶䴵刺槣칶&zip=z!z'sVf`JN&terms=\ued82㒛ꐏ馸駱",
 "/?item=ᗡ⏏㟗ᯣ&name=&email=댰鱔&city=鍎ႜ붠惬\uda92욗ﳹ\udeadϘ\ue967옮\uec2a聘㬃蜴퇌翗뚣뭶ໆ낢ܐ蛊&zip=uucF57]'ft&terms=둓ⓨ鶽糥",
 '/order?item=ﰮ믘聖垾&name=텼\uf817ه珝剴譐\ude8aⱘ&email=\uf6de⋲릅늄┰ꋃ䜍\uf365텃⸜\ueef9엘蠟ᆮ㌬澞寢&city=摫铝葳鐟嬀驲䰷풥ｓ\uf204뀽쁸&zip=yu;?Ms~;[S&terms=㢓弼',
 '/order?item=쿻&name=෯Ǥ∱컬䈁݇&email=⾞ꖦ鏋ⓩ맶\udda8뗢隿ⶮ◘蟜혵&city=霭ퟯǷ\u0a7b씀&zip=dco7`n;N%.&terms=鳹䡔鴗',
 '/order?item=\u0ad3쑝鐽囖恁ꖺ譕&name=䮌ṧꘋ죽ዥ\u4dbbꕑཆ嚍뎗᯳⮁睸烵\uf727퀘驃頽&email=逛ꇟ潑ﺷゞ녁\uf52aｇ镃\uf57b᭨ꐜ䍮跌\ud8f8ῆ\udc42⻱\ueeb2暟춘ƃ饣濐萓佷᭻&city=준筪\ued01&zﺻp=+1)FF_GdsI&terms=掌젪킼']

complex_population_5 = ['/order?item=盘ொ\udb99柋ꂌ\ue655name=璮卑ꌭ㵡\uef1cं┧樯❏骤셞乢玁뼁\ue214額㶅▶ő黕뫓隷䞺\ua9da\u09b1東물樯侤뫩덷竁Ⴆ闀鎼꙳ጪጃ减묆鐝⤎规email=䲩늫猭䭗\ud8ec鑆셡ԗ䊟city=ﶨ偏zip=J>f0-|"Vvjterms=ᴐ街\uedba➩\ue12f',
 '/order?item=\ue03bᦱ⍽㎉瓧&name=〡ⶒ虯밌\x0c椀풬嫤쌗ꔻ犔鲾\udca8参縹ഹᄌ癡쩚峻㻐屏壑玌ᖠ풀\ue5c7薯셴꺱찏\u0b00㘑ꊑ\ud928䱜钑璿勦䇲\ue558桌ູ쁮ᱴ\ue8e6\ued13勸&email=롚讍\uf69c천䞰꯵斛ڠ⊑ɝ꽔畐綎趝ᅨ変泙菑㯊빍詇\ueafb侫ᒉꃏ&\ueb55ity=᧼ꇢ7㜃\u181c\udd55肖⩴㨤Ｏ家껉筌㯋︳\uda7a嫃&zip=YWMYq"3zTT&terms=猱',
 "炾䨣ꖂ㶢&city=鰺謺㦐&zip=2'6;L#<a)]&terms=ꏗ/order?item=窬&name=巵疙ꀠ添丶ܹ搜ꀾ툛戬\ua9da㸏킆弼⽤㯘粋킴鍥䄌䑁㷫ꮓ讻\ua7f4澒殚ꢅ䩾㨵\u1774钡\uefe7鍞\udafb⠞馓\uf546꒗醶៨乢ﻼ&email=敝款\uf232訮繂캲\udb08䑜眧\ude68",
 '/order?item=溪&name=株蓨\u0bac\ue727ⱑ\ueec5࠱㬲ꤿᆓ붒ᕭ馅곎&email=祚殺茣ጒ䨄&city=',
 '/?item=ി&name=䳔忂\ued1cퟖ첺✠࿘氿悱䋌㸓馪\ueb02䟀㼲ඏ⌽끊䍽꠹㈄㎹챬⒜扂B跤䤲迨⫬㳕騪厊傚斻\ud8b8眽\uf559厯쀂쥍⦽苳뺐ꥆ㽓绚\ue938&email=ꔀ챉\uf06e齑抏ݢ홆\ud896䭵蔕Ӄ&city=鈿鴎&zip=SRAs}-]PqV&terms=焙\uee6eъ',
 '/order?item=뱓霰픷귝㇙&name=吔撪൯逸❡ꉈ꿧깂稝淰徕䉳\ueaed溾㦩뵭垽ヽ溚\ue273ꮋ㤉銺썠\udda4ౡ毐⨭\u31ea䙯툃㭱䲢霜&email=宐\uf195쳓๏ߢ並\uf64d嗚㕞\uf424釬\ue758뒮揽璺㜚湋&city=깎兒鵖￭镔饳紳蝔컦먹\ue18b䪔䠋⺊\ue19d&zip=Ew1(A8U/31&terms=땇',
 '/?item=&name=\uf6ca꼎卺Ը☤\uf496碮볘\uefb7珀㿵\udc9c\ua7e6恼%䳩ᔴ퀵⪧涶럐᳤Ὠ᷉珋쟾翍ᤜ♜ꏙ산\uf6b4\uf652뎧\ue2b2\ude78᳕跋岪覘ꕠ瓵\ue95a㰸\u087e눦\ud864&email=&city=圁鎤ທ䮐\ue134峑裕⽟뜊㊹턏\uf88a쬬곷ԟ❡䗷縳&zip=fqE?DbuyCK&terms=磫\ue177ꢬ絇\uf3b7',
 '/?item=薔ᾷ\ufdea偱볡&name=\uf4d4ᐏ㛀\ue856浮靈枕齱\uec20֖༿䳆З\udd61\ue90f諒玭&email=\u1ca8\uf135쓪ݶ뺀\uf484캍ꂿ挤Ќ\ue85fⴌ윳쳄ፚ⿺铕&city=鴔팹쑳槝\ue248嘎磒ഫ퓊＜&zip=Qf5<}&OUG[&terms=䊄\ue804㷁',
 '/order?item=ꂢ봉\uee72睐name=꜕暪ᗬ⧳믃㢓\uf33c맏菒螌䶘㹒쮭\uec5c讠ṋ㻓\udf78邝鬁疹뉱쭔İ\uf0de젫\uf27c䛯❓\uf12f⎼\u0bde\ue778앨뻁뤼詯ኋ鬩\ue232䤇嘕\uf0e2馏\udc80踩email=দ⻖㼹딊鱚럾휜슶浚俯頩ヰ퀠city=ﰆ띐\udbfcᴭʼ纛綵侼蘪zip=2/.cdl2~l>terms=〢쒼',
 '/?item=⇶퀍侵⠜䱤㧋岀ﻝ&name=疨\uf06b䃭\ueac2壻頧\u2d6d絗\uda83\udf6e䷓㡅&email=ƿ◇ᐞⓝ罆⺙蒛得\ue0c1\uec88䭌\uf61e\ue85a\uf3ec格&city=锜䂉\uedf9\u089dᴄ큨毯ꬥ⌞〘\uee00䯣梬蔔눍澐耯茖䑨孬&zip=5S8WT^Tsw9&terms=귨Ꝛ巣']


In [ ]:
prefix = ["easy", "medium", "hard", "complex"]

for i in range(1,6):
    for j in range(0,4):
        str_pop = prefix[j] + "_population_" + str(i)
        assert(len(eval(str_pop)) == 10), print("{0}: {1}".format(str_pop, len(eval(str_pop))))

In [ ]:
initial_population = [easy_population_1, easy_population_2, easy_population_3, easy_population_4, easy_population_5, \
              medium_population_1, medium_population_2, medium_population_3, medium_population_4, medium_population_5] 

harder_population = [hard_population_1, hard_population_2, hard_population_3, hard_population_4, hard_population_5, \
              complex_population_1, complex_population_2, complex_population_3, complex_population_4, complex_population_5]

## Scoring

For each URL input in the population and its corresponding target, your implementaton should generate a list of test inputs, __maximum of 10 inputs__. These inputs will be executed on the server and graded based on:

* Number of iterations of your GA algorithm (less is better)
* Number of reached target pages , i.e. error pages, confirmation page and page not found
* Number of resulting valid inputs for each target page.


__Note__
* __For each target, we expect at least _one_ valid generated input to pass the target, out of maximum 10 generated inputs__.
    
* __Your implementation will be tested with a _secret_ (possibly more complicated) population which may contain inputs other than the ones provided in the project.__ 

## Grading framework

For grading:
  * __To pass__, your implementation should produce valid inputs that reach the confirmation page, page not found and all error pages for the `easy_population` and `medium_population`.
  __Points__ will be assigned based on the number of reached pages, reaching the confirmation page, number of resulting valid inputs and the number of iterations of your GA algorithms. 
  * __To obtain extra points__, your implementation should produce valid inputs that reach the confirmation page, page not found and all error pages for all or some of the `hard_population`. 
  __Points__ will be assigned based on the number of reached pages, reaching the confirmation page, number of resulting valid inputs and the number of iterations of your GA algorithms.
  * __Bonus Points__, your implementation should produce valid inputs that reach the confirmation page, page not found and all error pages for all or some of the `complex_population`. 
  __Points__ will be assigned based on the number of reached pages, reaching the confirmation page, number of resulting valid inputs and the number of iterations of your GA algorithms.

In [ ]:
import multiprocessing
from multiprocessing import Process

def run_genetic_algorithm_limited_time(func, args_1, args_2, args_3, kwargs, time):
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    p = Process(target=func, args=(args_1, args_2, args_3, return_dict), kwargs=kwargs)
    p.start()
    p.join(time)
    if p.is_alive():
        p.terminate()
        return False

    return True, return_dict.values()

In [ ]:
def compute_results(population, num_regex):
    global httpd_process, httpd_url, list_regex
    resulting_population = [] 
    num_generations = 0
    num_valid, num_invalid, validity_score, reachability_score = 0,0,0,0
    
    reachability_list, validity_list = [],[] 
    gen_list, pass_list = [], []
    
    dash = '-' * 115
    print(dash)
    print('{:<8s}{:<8s}{:<10s}{:>8s}{:>12s}{:>12s}{:>18s}{:>12s}{:>20s}'.format("pop_id", "reg_id", "target", "#gens", "#val", "#inval", "reachability", "validity(%)", "Pass(1)/Fail(0)"))
    print(dash)
      
    itn = 0 
    
    reg_ind = num_regex
    list_regex = [itemList[reg_ind], name_regex[reg_ind], email_regex[reg_ind], city_regex[reg_ind], zip_regex[reg_ind], terms_regex]
    httpd_process, httpd_url = start_httpd(ProjectHTTPRequestHandler)

    for test_population in population:
        itn += 1
        for target in target_list:

            try:
                ret = run_genetic_algorithm_limited_time(genetic_algorithm, test_population, target, list_regex, {}, 3600) 
                if ret: 
                    resulting_population = ret[1][0][0]
                    num_generations = ret[1][0][1]
            except Exception as e:
                e.__traceback__ = None
                traceback.clear_frames(e.__traceback__)
                pass


            if len(resulting_population) <= 10:
                reachability_score, num_valid = get_web_reachability(resulting_population, target)
                num_invalid = len(test_population) - num_valid
                validity_score = (num_valid/len(test_population)) * 100  

                gen_list.append(num_generations)

                reachability_list.append(reachability_score)
                validity_list.append(validity_score)

                pass_list.append(bool(validity_score > 1))

                print('{:<8d}{:<8d}{:<10s}{:>8d}{:>10d}{:>12d}{:>16d}{:>16d}{:>16b}'\
                      .format(itn, reg_ind + 1, target, num_generations, num_valid, num_invalid, int(reachability_score), int(validity_score), \
                            validity_score > 1))
            else:
                raise ValueError("Number of generated inputs ({0}) is greater than 10 ".format(len(resulting_population)))
    httpd_process.terminate()

    total = len(population) * len(target_list)
    
    per_passed = (sum(pass_list)/total) * 100
    per_failed = ((total - sum(pass_list))/ total) * 100
    reachability = sum(reachability_list)
    avg_validity = (sum(validity_list)/ len(validity_list))
    avg_gens = sum(gen_list)/len(gen_list)

    return per_passed, per_failed, reachability, avg_validity, avg_gens, \
        total, len(reachability_list), validity_list, gen_list

In [ ]:
easy_population = [easy_population_1, easy_population_2, easy_population_3, easy_population_4, easy_population_5]
medium_population = [medium_population_1, medium_population_2, medium_population_3, medium_population_4, medium_population_5] 
hard_population = [hard_population_1, hard_population_2, hard_population_3, hard_population_4, hard_population_5]
complex_population = [complex_population_1, complex_population_2, complex_population_3, complex_population_4, complex_population_5]

In [ ]:
easy_per_passed_0, easy_per_failed_0, easy_reachability_0, easy_avg_validity_0, easy_avg_gens_0, \
easy_total_0, easy_len_reachability_list_0, easy_validity_list_0, easy_gen_list_0 = compute_results(easy_population, 0)

In [ ]:
print("EASY REGEX 1")
print("Percent passed: {0:.2f}%".format(easy_per_passed_0))
print("Percent failed: {0:.2f}%".format(easy_per_failed_0))
print("avg % validity: {0:.2f}%".format(easy_avg_validity_0))
print("avg gen_list: {0:.2f}".format(easy_avg_gens_0))
print("Reachability: {0:.2f}".format(easy_reachability_0))

In [ ]:
medium_per_passed_0, medium_per_failed_0, medium_reachability_0, medium_avg_validity_0, medium_avg_gens_0, \
medium_total_0, medium_len_reachability_list_0, medium_validity_list_0, medium_gen_list_0  = compute_results(medium_population, 0)

In [ ]:
print("MEDIUM REGEX 1")
print("Percent passed: {0:.2f}%".format(medium_per_passed_0))
print("Percent failed: {0:.2f}%".format(medium_per_failed_0))
print("avg % validity: {0:.2f}%".format(medium_avg_validity_0))
print("avg gen_list: {0:.2f}".format(medium_avg_gens_0))
print("Reachability: {0:.2f}".format(medium_reachability_0))

In [ ]:
hard_per_passed_0, hard_per_failed_0, hard_reachability_0, hard_avg_validity_0, hard_avg_gens_0, \
hard_total_0, hard_len_reachability_list_0, hard_validity_list_0, hard_gen_list_0  = compute_results(hard_population, 0)

In [ ]:
print("HARD REGEX 1")
print("Percent passed: {0:.2f}%".format(hard_per_passed_0))
print("Percent failed: {0:.2f}%".format(hard_per_failed_0))
print("avg % validity: {0:.2f}%".format(hard_avg_validity_0))
print("avg gen_list: {0:.2f}".format(hard_avg_gens_0))
print("Reachability: {0:.2f}".format(hard_reachability_0))

In [ ]:
complex_per_passed_0, complex_per_failed_0, complex_reachability_0, complex_avg_validity_0, complex_avg_gens_0, \
complex_total_0, complex_len_reachability_list_0, complex_validity_list_0, complex_gen_list_0  = compute_results(complex_population, 0)

In [ ]:
print("COMPLEX REGEX 1")
print("Percent passed: {0:.2f}%".format(complex_per_passed_0))
print("Percent failed: {0:.2f}%".format(complex_per_failed_0))
print("avg % validity: {0:.2f}%".format(complex_avg_validity_0))
print("avg gen_list: {0:.2f}".format(complex_avg_gens_0))
print("Reachability: {0:.2f}".format(complex_reachability_0))

In [ ]:
final_per_passed = (easy_per_passed_0 + medium_per_passed_0 + hard_per_passed_0 + complex_per_passed_0) / 4
final_per_failed = (easy_per_failed_0 + medium_per_failed_0 + hard_per_failed_0 + complex_per_failed_0) / 4
final_reachability = easy_reachability_0 + medium_reachability_0 + hard_reachability_0 + complex_reachability_0 
final_val_list = easy_validity_list_0 + medium_validity_list_0 + hard_validity_list_0 + complex_validity_list_0
final_avg_validity = sum(final_val_list)/len(final_val_list)
final_gen_list = easy_gen_list_0 + medium_gen_list_0 + hard_gen_list_0 + complex_gen_list_0
final_avg_gens = sum(final_gen_list)/len(final_gen_list)

In [ ]:
print("Percent passed: {0:.2f}%".format(final_per_passed))
print("Percent failed: {0:.2f}%".format(final_per_failed))

In [ ]:
print("avg % validity: {0:.2f}%".format(final_avg_validity))
print("avg gen_list: {0:.2f}".format(final_avg_gens))

In [ ]:
print("Reachability: {0:.2f}".format(final_reachability))